# The rise of diversity in metabolic platforms across the Candidate Phyla Radiation

Code supporting Jaffe et al., 2020.

In [ ]:
from ete3 import Tree
import matplotlib, re, os, glob, math, wget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from Bio import SeqIO, SearchIO
import seaborn as sns
import subprocess as sp
import ecopy as ep
from skbio.stats.ordination import pcoa
sns.set('notebook')
%matplotlib inline 

### Table of Contents

[Part 1:](#Part-1)
1.    [process and analyze metadata](#process-and-analyze-metadata)<br>
2.    [set up phylogenetic analysis](#set-up-the-phylogenetic-analysis)<br>
3.    [manual hmm curation](#manual-hmm-curation)<br>
4.    [analyze marker genes](#analyze-marker-gene-set)<br>
5.    [create final marker set](#create-final-marker-set)<br>
6.    [run initial trees](#run-initial-trees)<br>
7.   [run concatenated trees with outgroups](#run-concatenated-trees-with-outgroups)<br>
8.   [analyze concatenated trees](#analyze-concatenated-trees)<br>

[Part 2:](#Part-2)
1.   [create metabolic annotations](#create-metabolic-annotations)<br>
2.   [correlating metabolism + phylogeny](#correlating-metabolism-+-phylogeny)<br>
3.   [metabolic reference set](#building-a-metabolic-reference-set)<br>
4.   [create gene trees](#create-gene-trees)<br>
5.   [metabolic case studies](#metabolic-case-studies)<br>
6.   [trait depth + homoplasy](#trait-depth-+-homoplasy-metrics)<br>
7.   [miscellaneous](#miscellaneous)<br>

In [ ]:
# N.B. FILL THIS IN WITH PATH
# TO GITHUB REPO
rootdir = ""

## FILL THESE IN WITH PATHS TO
## DEPENDENCY PROGRAMS:

## MAFFT - https://mafft.cbrc.jp/alignment/software/
mpath = ""
## FASTTREEMP - http://www.microbesonline.org/fasttree/
ftpath = ""
## PULLSEQ - https://github.com/bcthomas/pullseq
pspath = ""
## BMGE - https://bmcevolbiol.biomedcentral.com/articles/10.1186/1471-2148-10-210
bmgepath = ""
## PRODIGAL - https://github.com/hyattpd/Prodigal
ppath = ""
## HMMER HMMSEARCH - http://hmmer.org/download.html
hpath = ""
## IQTREE - http://www.iqtree.org/
iqpath = ""
## PROTEIN CLUSTERING - https://github.com/raphael-upmc/proteinClusteringPipeline/tree/master/scripts
pcpath = ""

# Part 1

# process and analyze metadata

We start with the set of 991 dereplicated, quality filtered genomes.

### verify genome set

In [ ]:
# create a list of the dereplicated genomes
drep_genomes = []
for genome in glob.glob(rootdir + "/dereplicated_genomes/*"):
    drep_genomes.append(os.path.basename(genome).replace(".contigs.fa", ""))

In [ ]:
print("The below dataframe should contain %d metadata records." %(len(drep_genomes)))

### read in metadata

In [ ]:
final_org_table = pd.read_csv(rootdir + "final_org_table.tsv", sep="\t")

### create final taxonomy field

In [ ]:
fot = final_org_table
# rename cols
fot.columns = fot.columns.str.replace('RP Inventory (total: 55)', "rp_55", regex=False)
fot.columns = fot.columns.str.replace("BSCG Inventory (total: 51)", "bscg_51", regex=False)

In [ ]:
# read in cpr tax names to search for
with open(rootdir + "/cpr_names.txt", "r") as infile:
    tax_db = [line.strip() for line in infile.readlines()]
# remove generic CPR from here
tax_db = [item for item in tax_db if item != "CPR"]

# then define a function to search df tax strings
# make it degenerating - from most to least specific

def assign_tax(row):
    
    tax = "None"
    found = False
    # first split up tax string
    elements = row["taxonomy"].replace(" ", "").split(",")
    # then search elements for known taxonomies
    for element in elements:
        for entry in tax_db:
            if entry in element:
                tax = entry
                found=True
                break
        if found: break
    
    # if superphylum level, check bin name for phylum name
    found_again = False
    if (tax == "Parcubacteria") or (tax == "Microgenomates") or tax=="None":
        for entry in tax_db:
            if entry in row["name"]:
                tax = entry
                found_again = True
                break
                  
    return tax

In [ ]:
fot["final_tax"] = fot.apply(assign_tax, axis=1)
# at this point good to check assignments including Microgenomates, Parcubacteria, None
# make some manual edits
fot["final_tax"] = fot["final_tax"].replace({"WS6":"Dojkabacteria", "TM7":"Saccharibacteria", \
                    "SR1":"Absconditabacteria", "SM2F11":"Doudnabacteria", "Djokabacteria":"Dojkabacteria", \
                     "CPR02":"CPR1", "CPR03":"CPR3", "BD1-5": "Gracilibacteria"})

# set up the phylogenetic analysis

Now, with the genome set and metadata created, we can set up the preliminary phylogenetic analysis.

In [ ]:
def cmdir(path):
    
    if not os.path.isdir(path):
        os.mkdir(path)

### create protein database

In [ ]:
cmdir(rootdir + "/proteins/")
cmdir(rootdir + "/proteins/prodigal")

In [ ]:
with open(rootdir + "prodigal_wrapper.sh", "w") as wrapper:
    for genome in glob.glob(rootdir + "/dereplicated_genomes/*"):
        # if alternatively coded, repredict proteins with code 25
        if "Gracilibacteria" in genome or "SR1" in genome or "BD1-5" in genome:
            code = "25"
        else:
            code = "11"
        call = ppath + " -i " + genome + " -m -g " + code + " -a " + \
            rootdir + "/proteins/prodigal/" + os.path.basename(genome).replace(".fa",".faa")
        wrapper.write(call + "\n")

In [ ]:
# check that everything made it
count = 0
for protein_file in glob.glob(rootdir + "/proteins/prodigal/*"):
    count +=1
print("Protein files were transferred successfully for %d genomes." %(count))

In [ ]:
# modify UBA genomes contig names - non unique
for proteome in glob.glob(rootdir + "/proteins/prodigal/UBA*"):
    
    genome_name = os.path.basename(proteome).split(".")[0]
    # construct call to sed to modify- nb escape backslash
    call = "cat " + proteome + " | sed -r 's/>(.+)/>" + genome_name + "_\\1/' > " + proteome + ".mod"
    sp.call(call, shell=True)
    # remove originals
    call2 = "rm " + proteome
    sp.call(call2, shell=True)
    # rename modified to take their place
    call3 = "mv " + proteome + ".mod " + proteome
    sp.call(call3, shell=True)

In [ ]:
# create concatenated protein file for hmm
call = "cat " + rootdir + "/proteins/prodigal/* > " + rootdir + "/proteins/all_proteins.faa"
sp.call(call, shell=True)

### run hmms

In [ ]:
tigr_dict = {"rpol":{"TIGR02013":"rpoB", "TIGR02386": "rpoC"}, 
             "rp16":{"PF00410": "rpS8","PF00281":"rpL5","TIGR00060":"rpL18","TIGR01009":"rpS3",
            "TIGR01044":"rpL22","TIGR01049":"rpS10","TIGR01050":"rpS19","TIGR01067":"rpL14",
            "TIGR01071":"rpL15","TIGR01079":"rpL24","TIGR01164":"rpL16","TIGR01171":"rpL2",
            "TIGR03625":"rpL3","TIGR03635":"rpS17","TIGR03654":"rpL6","TIGR03953":"rpL4"}}

In [ ]:
def run_hmm(hmm_name):
    
    call = hpath + " --tblout " + rootdir + "/proteins/hmm_results/" + hmm_name + \
        ".results " + rootdir + "hmms/" + hmm_name + \
        ".HMM " + rootdir + "/proteins/all_proteins.faa"
    sp.call(call, shell=True)

In [ ]:
cmdir(rootdir + "/proteins/hmm_results/")

for key, items in tigr_dict.items():
    for hmm in items:
        run_hmm(hmm)

### parse results

In [ ]:
def parse_hmm(result_table):
    
    temp = {}
    count = 0
    
    # parse each result file using searchio
    for result in SearchIO.parse(result_table, "hmmer3-tab"):
        for item in result.hits:
            temp[count] = {"gene": item.id, "score": item.bitscore, "eval": item.evalue}
            count += 1
            
    return(pd.DataFrame.from_dict(temp, orient="index"))

In [ ]:
hmm_results = {}

for hmm_result in glob.glob(rootdir + "/proteins/hmm_results/*results"):
    hmm_results[os.path.basename(hmm_result).split(".")[0]] = parse_hmm(hmm_result)

### visualize and process results

In [ ]:
with open(rootdir + "/proteins/hmm_results/wrapper.sh", "w") as wrapper:
    
    for key in hmm_results.keys():
    
        table = hmm_results[key]
        table["position"] = table.index
        table["significant"] = table["eval"].apply(lambda x: x < 0.05)
        #table["logeval"] = table["eval"].apply(lambda x: math.log10(x + 1e-200))
        outdir_basename = rootdir + "/proteins/hmm_results/" + key

        ## PARSE OUT SIGNIFICANT HITS
        with open(outdir_basename + ".sighits.txt","w") as outfile:
            for idx,rowe in table[table["significant"]==True].iterrows():
                outfile.write(rowe["gene"] + "\n")

        call = pspath + " -n " + outdir_basename + ".sighits.txt -i " + \
            rootdir + "/proteins/all_proteins.faa > " + \
            outdir_basename + ".sighits.faa"
        sp.call(call,shell=True)

        ## NOW ALIGN + TREE BUILD
        call = mpath + " --thread 6 --reorder " + outdir_basename + ".sighits.faa > " + \
            outdir_basename + ".sighits.mafft"
        wrapper.write(call + "\n")

        # BUILD TREE
        call = ftpath + " " + outdir_basename + ".sighits.mafft > " + \
            outdir_basename + ".sighits.tre"   
        wrapper.write(call + "\n")

        # GENERATE ITOL DATA
        with open(outdir_basename + ".itol.txt", "w+") as itol:
            itol.write("DATASET_SIMPLEBAR\nSEPARATOR COMMA\nDATASET_LABEL,hmm_score\nSHOW_VALUE,1\nCOLOR,#ff0000\nDATA\n")
            for idx,rowe in table[table["significant"]==True].iterrows():
                itol.write(rowe["gene"] + "," + str(rowe["score"]) + "\n")

        #THEN PLOT
        plt.figure(figsize=(10,3))       
        sns.regplot("position", "score", data=table, fit_reg=False, scatter_kws={'s':60}, color="blue")
        plt.xlabel("rank")
        plt.xticks()
        ax2 = plt.twinx()
        ax2.set_ylim(0, 0.05)
        ax2.grid(False)
        sns.regplot("position", "eval", data=table, fit_reg=False, scatter_kws={'s':60}, ax=ax2, color="orange")
        plt.title("%s" %(key))
        plt.savefig(outdir_basename + ".png", format="png")   

Chmod +x wrapper.sh and add `export OMP_NUM_THREADS=6`. Then, based on these results, we can then set cutoffs for further analysis.

# manual hmm curation

With HMMs run, visualized, we now perform a manual hmm curation using the above figures and hmm scores mapped to tree.

### get hmm info

In [ ]:
def get_cutoffs(row, cutoff_type):
    
    cutoff = "none"
    basedir = rootdir + "/hmms/"
    # extract noise cutoff
    for line in open(basedir + row["hmm"] + ".HMM"):
        m = re.search("^" + cutoff_type + "\s+(\S+).+", line)
        if m:
            cutoff = float(m.group(1))
    return cutoff

In [ ]:
hmm_dict = pd.DataFrame.from_dict(tigr_dict, orient="columns")
hmm_dict = hmm_dict.reset_index().fillna("None")
hmm_dict.columns = ["hmm", "rpol", "rp16"]
hmm_dict["name"] = hmm_dict.apply(lambda x: x["rpol"] if x["rpol"] != "None" else x["rp16"], axis=1)
hmm_dict["set"] = hmm_dict.apply(lambda x: "rpol" if x["rpol"] != "None" else "rp16", axis=1)
hmm_dict = hmm_dict.drop(["rpol", "rp16"], axis=1)
hmm_dict["noise_cutoff"] = hmm_dict.apply(lambda x: get_cutoffs(x, "NC"), axis=1)

### filter hmm results

In [ ]:
manual_cutoffs = {"PF00281": "all_significant", "PF00410": 59.6, "TIGR00060": 39, "TIGR01009": 29, "TIGR01044": "all_significant", \
                 "TIGR01049": "all_significant", "TIGR01050": "all_significant", "TIGR01067": "all_significant", "TIGR01071": 29.4,
                 "TIGR01079": 23.3, "TIGR01164": "all_significant", "TIGR01171": "all_significant", "TIGR02013": 25.5, 
                 "TIGR02386": 18.1, "TIGR03625": "all_significant","TIGR03635": "all_significant", "TIGR03654": "all_significant", 
                 "TIGR03953": "all_significant"}

hmm_results_filt = {}
eff_cutoffs = {}

for key in hmm_results.keys():
    table = hmm_results[key]
    table["significant"] = table["eval"].apply(lambda x: x < 0.05)
    if manual_cutoffs[key] == "all_significant":
        hmm_results_filt[key] = table[table["significant"]==True]
        effective_cutoff = min(list(table[table["significant"]==True]["score"]))
    else:
        effective_cutoff = manual_cutoffs[key]
        # not inclusive - cutoff = highest outlier entry
        hmm_results_filt[key] = table[table["score"] > effective_cutoff]
    # save out effective cutoffs
    eff_cutoffs[key] = effective_cutoff

In [ ]:
cmdir(rootdir + "/proteins/hmm_curation/")

for key in hmm_results_filt.keys():
    
    # first create sequence name list
    filepath = rootdir + "/proteins/hmm_curation/" + key + ".filt.names"
    with open(filepath, "w") as outfile:
        for index, row in hmm_results_filt[key].iterrows():
            outfile.write(row["gene"] + "\n")
    
    # then send to pullseq
    call = pspath + " -n " + filepath + " -i " + rootdir + "/proteins/all_proteins.faa > " + \
        rootdir + "proteins/hmm_curation/" + key + ".filt.faa"
    sp.call(call, shell=True)

### revisualize - Supp. Fig 2

In [ ]:
# add effective cutoffs to df
hmm_dict["manual_cutoff"] = hmm_dict["hmm"].apply(lambda x: eff_cutoffs[x])

In [ ]:
# set hmms to plot
exemplars = ["TIGR01009", "TIGR02013"]

In [ ]:
cmdir(rootdir + "/figures/")

for key,row in hmm_dict.iterrows():
        
        if row["hmm"] in exemplars:
            
            table = hmm_results[row["hmm"]]
            table["position"] = table.index
            table["significant"] = table["eval"].apply(lambda x: x < 0.05)

            #THEN PLOT
            sns.set(font_scale=1.5)
            sns.set_style("ticks")
            plt.figure(figsize=(20,6)) 
            
            for cutoff in ["noise_cutoff", "manual_cutoff"]:
                plt.axhline(float(row[cutoff]), ls='--', color="grey")
                # position text with slight adjustments
                plt.text(int(max(table["position"]))*0.01,float(row[cutoff]) + int(max(table["score"]))*0.02, cutoff.replace("_", " ") + \
                             " at " + str(row[cutoff]), color="grey")
                
            sns.regplot("position", "score", data=table, fit_reg=False, scatter_kws={'s':40}, color="blue")
            plt.xlabel("rank")
            plt.xticks()
            ax2 = plt.twinx()
            ax2.set_ylim(0, 0.05)
            ax2.grid(False)
            sns.regplot("position", "eval", data=table, fit_reg=False, scatter_kws={'s':40}, ax=ax2, color="orange")
            plt.title("%s" %(row["hmm"]))
            outdir = rootdir + "/figures/" + row["hmm"] + "_results_annotated"
            plt.savefig(outdir + ".svg", format="svg")
            plt.show()

# analyze marker gene set

We have a manually curated marker gene set that can be further analyzed and refined, if necessary. First, let's attach metadata, which requires a scaf2bin file.

### create a scaf2bin for all genomes

In [ ]:
scaf2bin = {}

# iterate through genomes
for genome in glob.glob(rootdir + "/dereplicated_genomes/*"):
    
    genome_name = os.path.basename(genome).split(".")[0]
    #iterate through scaffolds
    for record in SeqIO.parse(open(genome, "r"), "fasta"):
        # modify scaffold names for UBA
        if "UBA" in genome_name:
            scaf_name = genome_name + "_" + record.description
        else:
            # get rid of junk
            m = re.search("(\S+).+", record.description)
            scaf_name = m.group(1)
        # then add to dict
        scaf2bin[scaf_name] = genome_name

### get gene info and attach metadata

In [ ]:
def scaffold(gene):
    if gene != "None":
        try: return re.search("(.+?)_[0-9]+$", gene).group(1)
        except: print(gene)

def retrieve_bin(scaffold, scaf_dict):
    try: return scaf_dict[scaffold]
    except: print("%s not found in scaf2bin!" %(scaffold))

In [ ]:
gene_lengths = {}
count=0
# read in gene lengths
for seq_file in glob.glob(rootdir + "/proteins/hmm_curation/*.filt.faa"):
    name = os.path.basename(seq_file).split(".")[0]
    for record in SeqIO.parse(open(seq_file,"r"), "fasta"):
        m = re.search("(\S+).+", record.description)
        gene_lengths[count] = {"gene": m.group(1), "len":len(record.seq), "hmm":name, "seq": str(record.seq)}
        count += 1

In [ ]:
# create df
len_df = pd.DataFrame.from_dict(gene_lengths, orient="index")
# attach scaffold name
len_df["scaffold"] = len_df["gene"].apply(scaffold)
# attach bin name
len_df["bin"] = len_df["scaffold"].apply(lambda x: retrieve_bin(x,scaf2bin))
merged = pd.merge(len_df, fot, left_on="bin", right_on="name")
# and subset
merged = merged[["gene","scaffold","hmm", "len", "bin", "final_tax", "rp_55", "bscg_51","seq"]]

### check gene lengths

In [ ]:
for key, items in tigr_dict.items():
    merged_subset = merged[merged.hmm.isin(items)]
    # and plot
    plt.figure(figsize=(16,6))
    sns.violinplot(x="hmm", y="len", data=merged_subset, palette="Set2")
    sns.stripplot(x="hmm", y="len", data=merged_subset, palette="Set2", jitter=True, size=2, linewidth=1, color="black")
    plt.ylabel("protein length (aa)")
    plt.show()

### analyze partial genes

Some genomes have marker genes that are split, possibly due to errors in assembly. How common is this problem? Some split genes will have multiple hmm hits on the same contig.

In [ ]:
def get_multiple(row):
    mult = False
    for i in range(1,len(row)):
        if row[i] > 1:
            mult=True
    return mult

In [ ]:
# calculate hmm hit counts by scaffold
partial = len_df[["gene", "len", "hmm"]]
partial["ones"] = 1
pp = partial.pivot("gene", "hmm", "ones")
pp = pp.fillna(0)
pp["gene"] = pp.index
pp["scaffold"] = pp["gene"].apply(scaffold)
pp= pp.drop("gene", axis=1)
ppg = pp.groupby("scaffold", as_index=False).sum()
ppg["mult"] = ppg.apply(get_multiple, axis=1)
# select scaffolds with multiple hits of a single hmm
mult = ppg[ppg["mult"] == True]
mult = mult.drop("mult", axis=1)
print("There are %d scaffolds with multiple copies of at least one of the marker genes in both sets." %(len(mult)))

# create final marker set

### editing split genes

In [ ]:
def get_strand(gene):
    try: return strand_dict[gene]
    except: print("%s not found in strand_dict!" %(gene))
        
def check_consecutive(gene_list):
    # get gene no's and sort
    ns = sorted([int(item.split("_")[-1]) for item in gene_list])
    # calculate consec
    diff = ns[-1] - ns[0]
    if diff == (len(ns)-1):
        return True
    else: return False

In [ ]:
# pull strandedness information
strand_dict = {}

for record in SeqIO.parse(open(rootdir + "/proteins/all_proteins.faa","r"), "fasta"):
    gene_name = record.description.split(" # ")[0]
    strand = record.description.split(" # ")[3]
    strand_dict[gene_name] = strand

In [ ]:
modified_genes = []
removed_genes = []

cmdir(rootdir + "/proteins/partials/")

for key, items in tigr_dict.items():
    
    for hmm in items: 
        
        # define metadata
        name = hmm
        out_name = hmm + ".modified.faa"
        
        # only do this for hmms with split scaf results
        if len(mult[mult[name]>1]) > 0:
            
            with open(rootdir + "/proteins/partials/" + out_name, "w") as outfile: 

                # iterate through scaffolds with multiple hits for hmm of interest
                for index, row in mult[mult[name]>1].iterrows():
                    # get sub table from merged with component genes
                    sub_table = merged[(merged.scaffold == row["scaffold"]) & (merged["hmm"] == name)]
                    # then iterate through those genes
                    strands = []
                    genes = []
                    for i, r in sub_table.iterrows():
                        genes.append(r["gene"])
                        # then get the strandedness
                        strand = get_strand(r["gene"])
                        strands.append(strand)
                    # only proceed if on one strand and if consecutive
                    if ((len(set(strands)) == 1) & (check_consecutive(genes)==True)):
                        # sort by gene number
                        genes.sort(key= lambda x: int(x.split('_')[-1]))
                        # if comp strand, reverse
                        if list(set(strands))[0] == "-1":
                            genes.reverse()
                        
                        count = 0
                        for gene in genes:
                            seq = sub_table[sub_table.gene==gene]["seq"].values[0]
                            if "XX" in seq:
                                if count == 0: #most upstream gene
                                    outfile.write(str(">" + gene + "\n"))
                                    # trim off post error
                                    outfile.write(sub_table[sub_table.gene==gene]["seq"].values[0].split("X")[0] + "\n")
                                    modified_genes.append(gene)
                                    new_seq = sub_table[sub_table.gene==gene]["seq"].values[0].split("X")[0]
                                else: # if downstream, drop
                                    removed_genes.append(gene)
                            count +=1

In [ ]:
print((removed_genes))

### revisit hmm results

In [ ]:
# keep track of which
# hmms got merged seqs
hmms_w_merged = []

for key in hmm_results_filt.keys():
    
    # add merged info to df
    table = hmm_results_filt[key]
    table["position"] = table.index
    table["merged"] = table["gene"].apply(lambda x: x in modified_genes)
    # now plot
    if len(table[table["merged"] == True]) > 0:
        hmms_w_merged.append(key)
        
        for variable in ["score"]:
            
            sns.set(font_scale=1.5)
            sns.set_style("ticks")
            sns.lmplot("position", variable, data=table, fit_reg=False, hue="merged",scatter_kws={'s':20}, palette="coolwarm", size = 3, aspect=3, legend=False)
            plt.title("%s %s" %(key, variable))
            #if variable == "eval":
                #plt.ylim([0,10e-20])
            plt.xlabel("rank")
            plt.show()

### edit the sequence set

In [ ]:
for key, item in tigr_dict.items():
    
    for hmm in item:
        
        # initialize final seq file
        filename = hmm + ".final.faa"
        with open(rootdir + "/proteins/partials/" + filename, "w") as outfile:
            
            if hmm in hmms_w_merged:
                # first add new sequences for those w/ merged
                for merged_seq in SeqIO.parse(open(rootdir + "/proteins/partials/" + hmm + ".modified.faa", "r"),"fasta"):
                    outfile.write(">" + merged_seq.description + "\n" + str(merged_seq.seq) + "\n")

            # then add old sequences, but not ones that were merged
            for old_seq in SeqIO.parse(open(rootdir + "/proteins/hmm_curation/" + hmm + ".filt.faa", "r"),"fasta"):
                # pull clean headers
                m = re.search("(\S+).+", old_seq.description)
                if m.group(1) not in modified_genes and m.group(1) not in removed_genes:
                    outfile.write(">" + old_seq.description + "\n" + str(old_seq.seq) + "\n")

### reattach metadata

In [ ]:
# allow for and in merged headers
def rescaffold(gene):
    if gene != "None":
        try: return re.search("(.+?)_[0-9and]+$", gene).group(1)
        except: print(gene)

In [ ]:
trimmed_genes = {}
count=0
# read in gene lengths
for seq_file in glob.glob(rootdir + "/proteins/partials/*.final.faa"):
    name = os.path.basename(seq_file).split(".")[0]
    for record in SeqIO.parse(open(seq_file,"r"), "fasta"):
        m = re.search("(\S+).*", record.description)
        trimmed_genes[count] = {"gene": m.group(1), "len": len(record.seq), "hmm":name, "seq": str(record.seq)}
        count += 1

In [ ]:
# create df
tdf = pd.DataFrame.from_dict(trimmed_genes, orient="index")
# attach scaffold name
tdf["scaffold"] = tdf["gene"].apply(rescaffold)
# attach bin name
tdf["bin"] = tdf["scaffold"].apply(lambda x: retrieve_bin(x, scaf2bin))
tm = pd.merge(tdf, fot, left_on="bin", right_on="name")
tms = tm[["bin", "gene","scaffold","hmm","len"]]

### deal with multiples

The general strategy here is to summarize the contents of each scaffold in the genome, undergo a selection process by which genes on the same scaffold are preferenced, then subsequently ones with more data (longer combined length).

<b> N.B. </b> Takes a few minutes to run.

In [ ]:
to_keep = []
count = 0

# iterate through genomes
for genome in list(set(tms["bin"])):
    
    count+=1
    print('Processing genome [%d]\r'%count, end="")
    
    # get dataframe subset and scaffold
    df = tms[tms.bin==genome]
    df["scaf"] = df["gene"].apply(rescaffold)
    scafs = {}
    # iterate through scaffolds represented and add gene name + length
    for key, row in df.iterrows():
        if row["scaf"] not in scafs:
            scafs[row["scaf"]] = {"name": row["scaf"], "genes": {row["hmm"]}, "total_seq": int(row["len"])}
        else:
            scafs[row["scaf"]]["genes"].add(row["hmm"])
            scafs[row["scaf"]]["total_seq"] += int(row["len"])
    
    # for a given genome, which genes are present across all scafs?
    all_genes = []
    for key in scafs.keys():
        all_genes += list(scafs[
            key]["genes"])
    
    # next, choose the marker genes
    scafs_list = [scafs[item] for item in scafs.keys()]
    # multiple sort, first on # genes on contig, then for total length 
    newlist = sorted(scafs_list, key=lambda x: (len(x["genes"]), x["total_seq"]), reverse=True) 
    found = []
    scafs2keep = []
    for scaf in newlist:
        # stop when you get all genes needed
        if len(set(found)) == len(set(all_genes)):
            break
        for gene in list(scaf["genes"]):
            if gene not in found:
                found.append(gene)
                scafs2keep.append(scaf["name"])
    
    # second pass - removing duplicates on same scaf
    dfs = df[df.scaffold.isin(scafs2keep)]
    final = {}
    for key, row in dfs.iterrows():
        if row["hmm"] not in final:
            final[row["hmm"]] = [{"name": row["gene"], "len": row["len"]}]
        else: final[row["hmm"]].append({"name": row["gene"], "len": row["len"]})
    for hmm in final.keys():
        # take one that's longest - most info content
        sorted_genes = sorted(final[hmm], key=lambda x: x["len"], reverse=True) 
        to_keep.append(sorted_genes[0]["name"])

In [ ]:
# now filter on the recovered genes
tms2 = tms[tms["gene"].isin(to_keep)]
tmsc2 = tms2.groupby("bin", as_index=False).aggregate({"gene":"count"})
tmsc2.sort_values("gene", ascending=False).head()

Now all bins that previously had multiple marker gene copies have just one for each.

### check final data matrix

Now that we've dealt with duplicate entries, we can pivot the dataset on bin name and re-attach to our metadata.

In [ ]:
# genes to keep are processed, but in the to_keep list
tmsf = tms[tms["gene"].isin(to_keep)]
# now iteratively add this info back to final org table
base = fot
for hmm in set(tmsf["hmm"]):
    df = tmsf[tmsf["hmm"] == hmm][["bin", "gene"]]
    df.columns = ["name", hmm]
    # important left join - keep all bins
    base = pd.merge(base,df, on="name", how="left")
base = base.fillna("None")

In [ ]:
def count_markers(row,dataset):
    count = 0
    for hmm in dataset:
        if row[hmm] != "None":
            count += 1
    return count

base = base.drop_duplicates()

# tabulate marker set completeness for each dataset
for dataset, items in tigr_dict.items():
    col_name = dataset + "_count"
    base[col_name] = base.apply(lambda x: count_markers(x, items.keys()), axis=1)

# run initial trees

Now it's time to run some preliminary trees.

### edit sequence set

In [ ]:
# get names of all genes
final_markers = []
for key, row in base.iterrows():
    for dataset, item in tigr_dict.items():
        for hmm in item:
            if row[hmm] != "None":
                final_markers.append(row[hmm])

In [ ]:
cmdir(rootdir + "/proteins/trimmed")

for dataset, item in tigr_dict.items():
    for hmm in item:
        # initialize final final seq file
        filename = hmm + ".trimmed.faa"
        with open(rootdir + "/proteins/trimmed/" + filename, "w") as outfile:
            # then add old sequences, but not ones that were filtered out
            for old_seq in SeqIO.parse(open(rootdir + "/proteins/partials/" + hmm + ".final.faa", "r"),"fasta"):
                # pull clean headers
                m = re.search("(\S+).*", old_seq.description)
                if m.group(1) in final_markers:
                    outfile.write(">" + old_seq.description + "\n" + str(old_seq.seq) + "\n")

### align and trim with bmge

In [ ]:
with open(rootdir + "/proteins/trimmed/align.sh", "w") as outfile:
    for file in glob.glob(rootdir + "/proteins/trimmed/*trimmed*"):
        name = file.replace("faa","mafft")
        call = mpath + " --thread 10 --retree 2 --reorder " + file + " > " + name
        #sp.call(call, shell=True)
        outfile.write(call + "\n")

In [ ]:
cmdir(rootdir+"/trees/prelim_trees")

for alignment in glob.glob(rootdir + "/proteins/trimmed/*mafft*"):
    name = alignment.replace("mafft","bmge.mafft")
    call = "java -jar " + bmgepath + " -i " + alignment + " -t AA -m BLOSUM30 -of " + name
    sp.call(call, shell=True)
    # copy over to tree building dir
    call2 = "cp " + name + " " + name.replace("proteins/trimmed", "trees/prelim_trees")
    sp.call(call2, shell=True)

### send single gene trees to cluster

N.B. Uses qsub, modify below command if necessary.

In [ ]:
for trimmed_alignment in glob.glob(rootdir + "/trees/prelim_trees/*bmge*"):

    basename = os.path.basename(trimmed_alignment).split(".")[0]
    outpath = rootdir + "/trees/prelim_trees/" + basename
    call = "echo '" + iqpath + " -s " + trimmed_alignment + " -m TEST -nt AUTO -st AA -bb 1500 -pre " + outpath + "' | qsub -V -N " + basename
    sp.call(call, shell=True)
    #print(call)

### modify treefiles

In [ ]:
# build name dict
name_dict = {}
for key, row in base.iterrows():
    for key,item in tigr_dict.items():
        for hmm in item:
            # small changes to match post bmge names
            #mod = row[hmm].replace(".", "_").replace("-", "_")
            mod = row[hmm]
            name_dict[mod] = row["final_tax"] + ":" + row["name"]

def rename_leaf(leaf):
    try: return name_dict[leaf]
    except: print("%s not found!" %(leaf))

In [ ]:
for prelim_tree in glob.glob(rootdir + "/trees/prelim_trees/*treefile*"):
    
    #read tree in
    t = Tree(prelim_tree)
    # simplify leaf names
    for leaf in t:
        #trim leaf name
        trim = leaf.name.split("___")[0]
        leaf.name = rename_leaf(trim)

    # now write out the tree
    t.write(outfile=prelim_tree.replace("treefile","renamed.treefile"),format=2)

# run concatenated trees with outgroups

Now it's time to add in other bacteria as outgroups.

In [ ]:
def is_cpr(header):
    
    found = False
    for taxon in list(set(base["final_tax"])):
        if taxon in header or "Saccharimonas" in header:
            found = True
    return found

### merge phylogenetic datasets

In [ ]:
# set source for genome info
in_wd = rootdir + "/bac175/"
# set source for outfiles
out_wd = rootdir + "/trees/bac175_outgroup/"
cmdir(out_wd)
prefix = "BAC175"

In [ ]:
with open(out_wd + "/align.sh", "w+") as outfile:
    
    for hmm in glob.glob(in_wd + "hmm_results/*faa"):
    
        # concatenate trimmed data with references
        basename = os.path.basename(hmm).split(".")[0]
        call = "cat " + hmm + " " + rootdir + "/proteins/trimmed/" + basename + \
            ".trimmed.faa > " + out_wd + basename + "." + prefix + ".concat.faa"
        sp.call(call, shell=True)

        # generate alignment call
        aln_name = out_wd + basename + "." + prefix + ".concat.mafft"
        call = mpath + " --thread 8 --retree 2 --reorder " + out_wd + basename + "." + prefix + ".concat.faa" + " > " + aln_name
        outfile.write(call + "\n")

In [ ]:
for alignment in glob.glob(out_wd + "*mafft*"):
    name = alignment.replace("mafft", "bmge.mafft")
    call = "java -jar " + bmgepath + " -i " + alignment + " -t AA -m BLOSUM30 -of " + name
    sp.call(call, shell=True)

In [ ]:
bac_scaf2bin = {}

for line in open(rootdir + "/bac175/bac_scaf2bin.txt").readlines():
    
    splt = line.strip().split("\t")
    bac_scaf2bin[splt[0]] = splt[1]

In [ ]:
# get info for genomes
results = {}

for result in glob.glob(in_wd + "/hmm_results/*faa*"):
    hmm = os.path.basename(result).split(".")[0]
    for record in SeqIO.parse(open(result), "fasta"):
        m = re.search("(\S+).*", record.description)
        binname = retrieve_bin(m.group(1), bac_scaf2bin)
        
        if binname not in results:
            results[binname] = {hmm: m.group(1)}
        else:
            results[binname][hmm] = m.group(1)

In [ ]:
outgroup_df = pd.DataFrame.from_dict(results, orient="index").fillna("None")

def count_markers(row,dataset):
    count = 0
    for hmm in dataset:
        if row[hmm] != "None":
            count += 1
    return count
        
# tabulate marker set completeness for each dataset
for dataset, items in tigr_dict.items():
    col_name = dataset + "_count"
    outgroup_df[col_name] = outgroup_df.apply(lambda x: count_markers(x, items.keys()), axis=1)

In [ ]:
# then filter out incomplete
odf = outgroup_df[(outgroup_df["rpol_count"] == 2) & (outgroup_df["rp16_count"] >= 8)]
odf["name"] = odf.index

# generate cols to keep
cols_to_keep = ["name", "rpol_count", "rp16_count"]
for dataset, items in tigr_dict.items():
    for key in items:
        cols_to_keep.append(key)
            
# merge cpr and outgroup
base_sub = base[cols_to_keep]
odf_sub = odf[cols_to_keep]
merged = pd.concat([base_sub, odf_sub])

In [ ]:
merged_seq = merged
aln_lens = {}

def get_sequence(gene, seq_dict):
    if gene=="None": 
        return "None"
    else:
        try: return seq_dict[gene]
        except: 
            print("%s not found!" %(gene))
            return "None"
            
# add sequences to merged df
for trimmed_alignment in glob.glob(out_wd + "*bmge*"):
    
    # first read in trimmed sequences
    temp_dict = {}
    for record in SeqIO.parse(open(trimmed_alignment, "r"), "fasta"):
        # pull clean headers
        m = re.search("(\S+).*", record.description)
        temp_dict[m.group(1)] = str(record.seq)
    # now add to the dataframe using apply
    hmm = os.path.basename(trimmed_alignment).split(".")[0]
    col_name = hmm + "_seq"
    merged_seq[col_name] = merged_seq[hmm].apply(lambda x: get_sequence(x, temp_dict))
    # get aln len to use later
    aln_lens[hmm] = len(record.seq)

In [ ]:
# add back some metadata
base_sub = base[["name", "final_tax"]]
merged_meta = pd.merge(merged_seq, base_sub, on="name", how="left").fillna(False)
# add tax for chloroflexi
merged_meta["final_tax"] = merged_meta.apply(lambda x: x["final_tax"] if x["final_tax"] != False else prefix, axis=1)

### process phylogenetic outliers

Manually compile bins to keep based on the following criteria - clades > 1 member conserved in both previous concat trees, or known undersampled lineages (ie, Schleper or Torok).

In [ ]:
# read in problematic bins for each tree
pbins = {}
for file in glob.glob(rootdir + "/trees/RP*outliers*"):
    print(file)
    name = os.path.basename(file).split("_")[0]
    pbins[name] = []
    for line in open(file).readlines():
        pbins[name].append("_".join(line.split("_")[1:]).strip())

In [ ]:
def get_outlier(name):
    
    found = []
    for dataset in pbins:
        if name in pbins[dataset]:
            found.append(dataset)
    
    if len(found) > 1:
        return "BOTH"
    elif len(found) == 1:
        return found[0]
    else: return False
    
# add info in
merged_meta["outlier"] = merged_meta.apply(lambda x: get_outlier(x["name"]), axis=1)

In [ ]:
merged_meta["outlier"].value_counts()

### run concat trees with outgroups

In [ ]:
count_mins = {"rp16": 8, "rpol": 2}

# finally write out the concatenated alignment, pruning previously identified taxa as before
for dataset, items in tigr_dict.items():
    
    filename = out_wd + dataset + "_concat." + prefix + ".pruned.mafft"
    with open(filename, "w") as outfile:

        # for each genome meeting criteria
        for key, row in merged_meta.iterrows():
            # write genome if not outlier for this dataset, also meets min gene count threshes
            if (row[dataset + "_count"] >= count_mins[dataset]) and \
                (row["outlier"] != dataset.upper() and row["outlier"] != "BOTH"):
                # simplify name ahead of time
                seq_name = row["final_tax"] + "_" + row["name"]
                outfile.write(">" + seq_name + "\n")
                # now write out sequences
                for hmm in items:
                    col_name = hmm + "_seq"
                    # if missing gene, just add gaps
                    if row[col_name] == "None":
                        outfile.write("-"*aln_lens[hmm])
                    # if gene present
                    else:
                        outfile.write(row[col_name])
                outfile.write("\n")
            #else:
                #print(row["name"], row["outlier"], dataset, row[dataset + "_count"])

N.B. Uses qsub, modify below command if necessary.

In [ ]:
# and run the trees
for concat_align in glob.glob(out_wd + "*.pruned.mafft*"):
    
    basename = os.path.basename(concat_align).split(".")[0] + "." + prefix + ".pruned"
    call = "echo '" + iqpath + " -s " + concat_align + " -m MFP -st AA -bb 1500 -nt 48 -pre " + concat_align.split(".")[0] + "." + prefix + ".final.pruned' | qsub -V -N " + basename
    sp.call(call, shell=True)
    #print(call)

# analyze concatenated trees

Look at broader phylogenetic groupings and alpha diversity within groups.

### taxonomy overrides

In [ ]:
# read in file
overrides={}
for line in open(rootdir + "/trees/tax_override.txt","r").readlines():
    genome_name = "_".join(line.split(",")[0].split("_")[1:])
    overrides[genome_name] = line.split(",")[1].strip()
    
# add to base df
base["revised_tax"] = base.apply(lambda x: overrides[x["name"]] if x["name"] in overrides else x["final_tax"], axis=1)
base["override"] = base.apply(lambda x: x["name"] in overrides, axis=1)

### tree collapsing

In [ ]:
revised_tax = {}
for key, row in base.iterrows():
    revised_tax[row["name"]] = row["revised_tax"]

In [ ]:
def collapse_tree(treefile, dictionary):
    
    # first read in tree
    t = Tree(treefile)
    # get leaf names
    node2labels = t.get_cached_content(store_attr="name")

    # define func to assign category
    def search_tax(genome):
        try: return dictionary[genome]
        except: return "Not found"
    
    # define func to identify subtrees
    def processable_node(node):
        # use post override taxonomies
        taxes = [search_tax("_".join(item.split("_")[1:])) for item in list(node2labels[node])]
        if len(set(taxes)) == 1:
            return True
        else:
            return False

    # initialize itol outfiles
    name = os.path.basename(treefile).split(".")[0]
    outfile1 = open(treefile.replace(".treefile", ".collapse.txt"), "w")
    outfile1.write("COLLAPSE\nDATA\n")
    outfile2 = open(treefile.replace(".treefile", ".label.txt"),"w")
    outfile2.write("DATASET_TEXT\nSEPARATOR COMMA\nDATASET_LABEL,labels\nCOLOR,#ff0000\nSHOW_INTERNAL,1\nDATA\n")

    # iterate through subtrees of same taxonomy
    for subtree in t.iter_leaves(is_leaf_fn=processable_node):
        to_collapse = []
        # collect leaves
        for leaf in subtree:
            to_collapse.append(leaf.name)
        # don't want to collapse singletons
        if len(to_collapse) > 1:
            
            # itol only takes 2 nodes to collapse
            distances = []
            for leaf in subtree:
                for leaf2 in subtree:
                    distances.append({"distance":t.get_distance(leaf,leaf2), "nodes": [leaf.name,leaf2.name]})
            # so find the two farthest apart
            distances_sorted = sorted(distances, key= lambda x: x["distance"], reverse=True)
            tokens = distances_sorted[0]["nodes"]
            
            # now write out to files
            clade_name = search_tax("_".join(to_collapse[0].split("_")[1:])) + " (%d)" %(len(to_collapse))
            #first write collapse file
            outfile1.write("|".join(tokens) + "\n")
            # then write label file
            outfile2.write("|".join(tokens) + "," + clade_name + ",-1,#ff0000,bold,2,0\n")

    outfile1.close()
    outfile2.close()

In [ ]:
for treefile in glob.glob(rootdir + "/trees/bac175_outgroup/*final.pruned.treefile*"):
    #print(treefile)
    collapse_tree(treefile, revised_tax)

### phylogenetic groupings - fig 1a

In [ ]:
tokeep = ["Dojkabacteria", "WS6","WWE3", "Katanobacteria", "Saccharibacteria", "Howlettbacteria", \
    "Berkelbacteria", "Kazan", "Peregrinibacteria", "Gracilibacteria", "Absconditabacteria"]
tax2color = {"Microgenomates":'#b3e2cd',"Parcubacteria_1":'#fdcdac',"Parcubacteria_2":'#cbd5e8',"Parcubacteria_3":'#f4cae4',
             "Parcubacteria_4":'#e6f5c9',"Katanobacteria (WWE3)":'#fff2ae',"Saccharibacteria":'#f1e2cc',"Peregrinibacteria":'#cccccc',"other":"white"}

In [ ]:
# create slighly different mapping for downstream
mapping = {}
for line in open(rootdir + "/trees/tree_order.txt").readlines():
    phylum, group = line.strip().split(",")
    if group=="":
        group = "other"
    mapping[phylum] = group

In [ ]:
cmdir(rootdir + "/metabolism/itol/")

In [ ]:
def process_tree(treefile, dictionary, mode):
    
    # first read in tree
    t = Tree(treefile)
    # reroot on bac175
    outs = [leaf.name for leaf in t if "BAC175" in leaf.name]
    root_node = t.get_common_ancestor(outs)
    t.set_outgroup(root_node)
    # get leaf names
    node2labels = t.get_cached_content(store_attr="name")
    print("Tree read and nodes extracted.")
    
    # define func to assign category
    def search_tax(genome):
        try: return dictionary[genome] if mode == "div" else mapping[dictionary[genome]]
        except: return "Not found"
    
    # define func to identify subtrees
    def processable_node(node):
        # use post override taxonomies
        taxes = [search_tax("_".join(item.split("_")[1:])) for item in list(node2labels[node])]
        if len(set(taxes)) == 1:
            return True
        else:
            return False
    
    if mode == "phy":
        # initialize itol outfiles
        name = os.path.basename(treefile).split(".")[0]
        outfile1 = open(rootdir + "/metabolism/itol/" + name + ".allmeta.ranges.txt", "w")
        outfile1.write("TREE_COLORS\nSEPARATOR TAB\nDATA\n")
        print("file initialized...")
    
    count = 0
    # iterate through subtrees of same taxonomy
    for subtree in t.iter_leaves(is_leaf_fn=processable_node):
        
        to_collapse = []
        # collect leaves
        for leaf in subtree:
            to_collapse.append(leaf.name)
        #print(to_collapse)
        # don't want to collapse singletons
        if len(to_collapse) > 1:
            
            # itol only takes 2 nodes to collapse
            distances = []
            for leaf in subtree:
                for leaf2 in subtree:
                    # don't compare same nodes
                    if leaf.name != leaf2.name:
                        distances.append({"distance":t.get_distance(leaf,leaf2), "nodes": [leaf.name,leaf2.name]})
            # so find the two farthest apart
            distances_sorted = sorted(distances, key= lambda x: x["distance"], reverse=True)
            tokens = distances_sorted[0]["nodes"]
            
            # now create output
            clade_name = search_tax("_".join(to_collapse[0].split("_")[1:]))
            
            if mode == "phy":
                if clade_name != "other":
                    #first write collapse file
                    try: col = tax2color[clade_name]
                    except: col = "white"
                    outfile1.write("|".join(tokens) + "\trange\t" + col + "\t" + clade_name + "\n")
                    # then write label file
                    #outfile2.write("|".join(tokens) + "," + clade_name + ",-1,#ff0000,bold,2,0\n")
            count +=1
            #print("%d subtrees processed..." %(count))
    
    if mode == "phy":
        outfile1.close()
        #outfile2.close()

In [ ]:
for treefile in glob.glob(rootdir + "/trees/bac175_outgroup/rp16*final.pruned.treefile"):
    process_tree(treefile, revised_tax, "phy")
    print(treefile)

# Part 2

Back to [top](#Table-of-Contents).

# create metabolic annotations

### create metadata and hmm files

In [ ]:
# read in hmm metadata
hmm_metadata = pd.read_csv(rootdir + "/metabolism/hmms/cpr_metabolism_table - hmm_subset_public.tsv", sep="\t").fillna("none")
# remove parsing error
hmm_metadata["hmm_name"] = hmm_metadata["hmm_name"].apply(lambda x: x.replace("\xa0",""))
hmm_metadata.head()

In [ ]:
def get_cutoffs(row, cutoff_type):
    
    cutoff = "none"
    if row["source"] in ["Pfam","TIGRFAMs"]:
        # get hmm location
        basedir = rootdir + "/metabolism/hmms/by-source/" + row["source"] + "/"
        # extract noise cutoff
        for line in open(basedir + row["hmm_name"] + ".hmm"):
            m = re.search("^" + cutoff_type + "\s+(\S+).+", line)
            if m:
                cutoff = float(m.group(1))
    return cutoff

In [ ]:
# now fill in noise and trusted cutoffs from hmm files
hmm_metadata["noise_cutoff"] = hmm_metadata.apply(lambda x: get_cutoffs(x, "NC"), axis=1)
hmm_metadata["trusted_cutoff"] = hmm_metadata.apply(lambda x: get_cutoffs(x, "TC"), axis=1)

### now actually run hmms

In [ ]:
cmdir(rootdir + "/metabolism/hmm_results/")

In [ ]:
def run_metabolic_hmm(hmm_name):
    
    basename = os.path.basename(hmm_name).replace(".hmm", "")
    result_file = rootdir + "/metabolism/hmm_results/" + basename + ".results"
    call = hpath + " --cpu 10 --tblout " + result_file + " " + hmm_name + " " + rootdir + "/proteins/all_proteins.faa"
    sp.call(call, shell=True)

In [ ]:
total = 1
for source in glob.glob(rootdir + "metabolism/hmms/by-source/*"):
    for hmm in glob.glob(source + "/*"):
        run_metabolic_hmm(hmm)
        print('hmms run: [%d]\r'%total, end="")
        total += 1

### prepare file hierarchy

In [ ]:
# create informative file hierarchy for hmm data products
basepath = rootdir + "/metabolism/hmm_data_products/"

if not os.path.exists(basepath):
    # create top directory
    os.mkdir(basepath)
    # then create subdirs for each process
    for supergroup in list(set(hmm_metadata["supergroup"])):
        os.mkdir(basepath + supergroup.replace(" ", "_") + "/")
        for group in list(set(hmm_metadata[hmm_metadata["supergroup"]==supergroup]["group"])):
            scrubbed_name = group.replace(" ","_").replace("/", "_")
            os.mkdir(basepath + supergroup.replace(" ", "_") + "/" + scrubbed_name)

### create hmm data products for analysis

In [ ]:
## READ IN RESULTS

metabolic_hmm_results = {}

for hmm_result in glob.glob(rootdir + "/metabolism/hmm_results/*"):
    metabolic_hmm_results[os.path.basename(hmm_result).split(".")[0]] = parse_hmm(hmm_result)

In [ ]:
# SET OPTIONS
sns.set_style("ticks")
## open bash wrapper
wrapper = open(rootdir + "/metabolism/hmm_data_products/wrapper.sh", "w+")
total=1

### CREATE HMM DATA PRODUCTS
for index, row in hmm_metadata.iterrows():
    
    key = row["hmm_name"]
    try:
        table = metabolic_hmm_results[key]
    except: # if no matching hmm
        continue
    
    #only process if there are > x results
    if len(table) > 0:
        
        ## PROCESS TABLE + METADATA
        
        table["position"] = table.index
        # evaluate score and eval
        table["significant"] = table["eval"].apply(lambda x: x < 0.05)   
        # define outdir for data products
        scrubbed_name = row["group"].replace(" ","_").replace("/", "_")
        outdir_basename = rootdir + "/metabolism/hmm_data_products/" + \
            row["supergroup"].replace(" ", "_") + "/" + scrubbed_name + "/" + key
    
        ## FIRST, PLOT
        
        fig = plt.figure(figsize=(20,4))
        # plot trusted and noise cutoffs if applicable
        for cutoff in ["noise_cutoff", "trusted_cutoff"]:
            if row[cutoff] != "none":
                plt.axhline(float(row[cutoff]), ls='--', color="grey")
                # position text with slight adjustments
                plt.text(int(max(table["position"]))*0.93,float(row[cutoff]) + int(max(table["score"]))*0.025, cutoff + \
                         " cutoff at " + str(row[cutoff]), color="grey")
        # plot published cutoff if nothing else (custom hmms)
        if row["noise_cutoff"] == "none" and row["trusted_cutoff"] == "none" and row["published_cutoff"] != "none":
            plt.axhline(float(row["published_cutoff"]), ls='--', color="grey")
            plt.text(int(max(table["position"]))*0.93,float(row["published_cutoff"]) + int(max(table["score"]))*0.025, "published" + \
                     " cutoff at " + str(row["published_cutoff"]), color="grey")

        # plot actual data
        sns.regplot("position", "score", data=table, fit_reg=False, scatter_kws={'s':80}, color="blue")
        plt.xlabel("rank")
        plt.xticks()
        ax2 = plt.twinx()
        ax2.set_ylim(0, 0.05)
        ax2.grid(False)
        sns.regplot("position", "eval", data=table, fit_reg=False, scatter_kws={'s':80}, ax=ax2, color="orange")
        plt.title("%s (%s), published cutoff: %s" %(row["hmm_name"], row["gene"], row["published_cutoff"]))
        plt.savefig(outdir_basename + ".png", format="png")
        # don't display plot
        plt.close(fig)
        
        ## FOR HMMS WITH SIG RESULTS
        if list(table["significant"]).count(True) > 0:
            
            ## PARSE OUT SIGNIFICANT HITS
            with open(outdir_basename + ".sighits.txt","w") as outfile:
                for idx,rowe in table[table["significant"]==True].iterrows():
                    outfile.write(rowe["gene"] + "\n")
            
            call = pspath + " -n " + outdir_basename + ".sighits.txt -i " + \
                rootdir + "/proteins/all_proteins.faa > " + \
                outdir_basename + ".sighits.faa"
            sp.call(call,shell=True)
            
            ## NOW ALIGN + TREE BUILD
            if list(table["significant"]).count(True) > 1:
                call = mpath + " --thread -10 --reorder " + outdir_basename + ".sighits.faa > " + \
                    outdir_basename + ".sighits.mafft"
                wrapper.write(call + "\n")
            
            if list(table["significant"]).count(True) > 5:
                
                # BUILD TREE
                call = ftpath + " " + outdir_basename + ".sighits.mafft > " + \
                    outdir_basename + ".sighits.tre"   
                wrapper.write(call + "\n")
                
                # GENERATE ITOL DATA
                with open(outdir_basename + ".itol.txt", "w+") as itol:
                    itol.write("DATASET_SIMPLEBAR\nSEPARATOR COMMA\nDATASET_LABEL,hmm_score\nSHOW_VALUE,1\nCOLOR,#ff0000\nDATA\n")
                    for idx,rowe in table[table["significant"]==True].iterrows():
                        itol.write(rowe["gene"] + "," + str(rowe["score"]) + "\n")
        
        print('hmms processed: [%d]\r'%total, end="")
        total += 1

wrapper.close()

<b> Then chmod +x, run 'export OMP_NUM_THREADS=6' for FastTreeMP and run the wrapper manually. </b>

### run extra domain hmms for tricky cases

In [ ]:
domains = {"TIGR01418": ["PF01326", "PF00391", "PF02896"], "TIGR01251": ["PF13793", "PF14572"]}

In [ ]:
cmdir(rootdir + "/metabolism/extra_markers")

In [ ]:
inpath = rootdir + "/metabolism/hmm_data_products/" + \
    "Higher_C_compounds_metabolism/"
slugs = {"TIGR01418": "Pyruvate_metabolism/","TIGR01251":"Pentose_Phosphate_Pathway:_Non-Oxidative_Branch/"}
outpath = rootdir + "/metabolism/extra_markers/"

for enzyme, pfams in domains.items():
    
    in_faa = inpath + slugs[enzyme] + enzyme + ".sighits.faa"
    pfam_dfs = {}
    
    for pfam in pfams:
        
        #pull down pfam
        url = "https://pfam.xfam.org/family/%s/hmm" %(pfam)
        # name according to pfam, otherwise gene name
        outfile = outpath + pfam + ".hmm"
        wget.download(url, out=outfile)
        
        #run pfam
        call = hpath + " --cut_nc --cpu 6 --tblout " + outpath + pfam + ".results " + outpath + pfam + ".hmm " + in_faa
        sp.call(call, shell=True)
        
        # read in results
        result = parse_hmm(outpath + pfam + ".results")
        
        # then write out to itol
        with open(outpath + pfam + ".extra.itol.txt", "w") as outfile:
        
            outfile.write("DATASET_BINARY\nSEPARATOR COMMA\nDATASET_LABEL," + pfam + "\nCOLOR,#ff0000\nFIELD_SHAPES,2\nFIELD_LABELS,")
            outfile.write(pfam + "\n")
            outfile.write("FIELD_COLORS,#000000\nDATA\n")
            
            for key, row in result.iterrows():
                outfile.write(row["gene"] + "," + "1" + "\n")        

### filter, select, merge results

In [ ]:
cutoffs = {}

for index,row in hmm_metadata.iterrows():
    if row["hmm_name"] != "none":
        man = row["manual_cutoff"]
        if man != "none":
            if man == "noise":
                man = row["noise_cutoff"]
            elif man == "trusted":
                man = row["trusted_cutoff"]
            elif man == "published":
                man = row["published_cutoff"]
            elif man == "all_significant":
                man = 0.05
            else: man = float(man)
            
            cutoffs[row["hmm_name"]] = man

In [ ]:
# then get filtered results
metabolic_hmm_results_filtered = {}

for key in metabolic_hmm_results.keys():
    # get cutoff - first try new then old
    try:
        cutoff = cutoffs[key]
    except:
        print(key + " not processed.")
        continue
    
    #print(key, cutoff)
    table = metabolic_hmm_results[key]
    
    # only do if there are results and a cutoff
    if len(table) > 0:
        # score cutoff (non -inclusive)
        if cutoff > 1:
            table_filt = table[table["score"] > cutoff]
        # eval cutoff (non -inclusive)
        else:
            table_filt = table[table["eval"] < cutoff]

        table_subset = table_filt[["gene", "score","eval"]]
        table_subset["hmm"] = key
        metabolic_hmm_results_filtered[key] = table_subset

In [ ]:
#concatenate all hmms
all_results = pd.concat(list(metabolic_hmm_results_filtered.values()))
# then recast as long
all_results_long = all_results.pivot("gene", "hmm", "score").fillna(0)
# select best hit per orf
all_results_long["best_hmm"] = all_results_long.idxmax(axis=1)
all_results_long["best_score"] = all_results_long.max(axis=1)
all_results_sub = all_results_long.reset_index()[["gene", "best_hmm", "best_score"]]

### then finish analysis

In [ ]:
all_results_sub["scaf"] = all_results_sub["gene"].apply(scaffold)
all_results_sub["bin"] = all_results_sub["scaf"].apply(lambda x: retrieve_bin(x, scaf2bin))
hmms_df = all_results_sub[["bin", "gene", "best_hmm", "best_score"]]
# select best hit for each hmm within a bin
hmms_dfp = pd.pivot_table(hmms_df,index=["bin","best_hmm"],columns="gene", values="best_score").fillna(0)
hmms_dfp["best_gene"] = hmms_dfp.idxmax(axis=1)
hmms_dfp = hmms_dfp.reset_index()[["bin", "best_hmm", "best_gene"]]

In [ ]:
# pivot one more time
hmms_final = hmms_dfp.pivot("bin", "best_hmm", "best_gene").fillna("None")
hmms_final = hmms_final.reset_index()
# merge
base_hmms = pd.merge(base, hmms_final, left_on="name", right_on="bin", how="left").fillna("None")

In [ ]:
def merge_columns(row, input_cols):
    
    gene_list = []
    
    for col in input_cols:
        
        if row[col] != "None":
            gene_list.append(row[col])
    
    # return first hit
    if len(gene_list) > 0:
        return gene_list[0]
    else:
        return "None"

In [ ]:
merged_traits = {}

for key, row in pd.DataFrame(hmm_metadata["gene"].value_counts()).iterrows():
    
    # for traits w/ multi hmms
    if row.iloc[0,] > 1:
        
        # which cols are actually included
        cols = set(hmm_metadata[hmm_metadata["gene"]== key]["hmm_name"].to_list()).intersection(list(base_hmms.columns))
        
        # only for those with hits
        if len(cols) > 0:
            merged_traits[key + "_merged"] = list(cols)
            base_hmms[key + "_merged"] = base_hmms.apply(lambda x: merge_columns(x, list(cols)), axis=1)

### revisualize hmm cutoffs - supp. fig. 3

In [ ]:
exemplars = ["TIGR00419", "PF00316"]

In [ ]:
for key,row in hmm_metadata.iterrows():
        
        if row["hmm_name"] in exemplars:
            
            table = metabolic_hmm_results[row["hmm_name"]]
            table["position"] = table.index
            table["significant"] = table["eval"].apply(lambda x: x < 0.05)

            #THEN PLOT
            sns.set(font_scale=1.5)
            sns.set_style("ticks")
            plt.figure(figsize=(20,6)) 
            
            for cutoff in ["noise_cutoff", "manual_cutoff"]:
                plt.axhline(float(row[cutoff]), ls='--', color="grey")
                # position text with slight adjustments
                plt.text(int(max(table["position"]))*0.01,float(row[cutoff]) + int(max(table["score"]))*0.02, cutoff.replace("_", " ") + \
                             " at " + str(row[cutoff]), color="grey")
                
            sns.regplot("position", "score", data=table, fit_reg=False, scatter_kws={'s':40}, color="blue")
            plt.xlabel("rank")
            plt.xticks()
            ax2 = plt.twinx()
            ax2.set_ylim(0, 0.05)
            ax2.grid(False)
            sns.regplot("position", "eval", data=table, fit_reg=False, scatter_kws={'s':40}, ax=ax2, color="orange")
            #plt.title("%s" %(row["hmm_name"]))
            outdir = rootdir + "/figures/" + row["hmm_name"] + "_results_annotated"
            plt.savefig(outdir + ".svg", format="svg")
            plt.show()

# correlating metabolism + phylogeny

### basic trait distribution

In [ ]:
merged_traits_values = [item for sublist in merged_traits.values() for item in sublist]

In [ ]:
# make hmm metadata accessible for sorting and labelling
groups = {}
for key,row in hmm_metadata.iterrows():
    
    if row["hmm_name"] in merged_traits_values:
        groups[row["gene"] + "_merged"] = {"group":row["supergroup"], "process": row["group"], "name": row["gene"]}
    else:
        groups[row["hmm_name"]] = {"group":row["supergroup"], "process": row["group"], "name": row["gene"]}

def get_info(x, datum):
    try: return groups[x][datum]
    except: return x

In [ ]:
# get traits and sort them
traits =[trait for trait in list(set(hmms_df["best_hmm"])) if trait not in merged_traits_values] + list(merged_traits.keys())
traits_sorted = sorted(traits, key=lambda x: (get_info(x, "group"),get_info(x, "process")))
# create new name to match tree
base_hmms_g = base_hmms[["name","final_tax","revised_tax"] + traits_sorted].drop_duplicates()

#reformat dataframe for heatmap
base_hmms_clean = base_hmms_g[["name", "revised_tax"]]

total = 0
for trait in traits_sorted:
    if len(set(base_hmms_g[trait])) != 1:
        base_hmms_clean[trait] = base_hmms_g[trait].apply(lambda x: 1 if x != "None" else 0)
    
    print('traits processed: [%d]\r'%total,end="")
    total += 1

#define agg dict
agg_dict = {}
for trait in traits_sorted:
    agg_dict[trait] = "sum"
agg_dict["name"] = "count"

#reformat data by phylum
base_hmms_gb = base_hmms_clean.groupby("revised_tax",as_index=False).aggregate(agg_dict)
# then normalize counts by phylum size
for trait in traits_sorted:
    base_hmms_gb[trait] = base_hmms_gb.apply(lambda x: x[trait]/float(x["name"]), axis=1)
base_hmms_gb.pop("name")
base_hmms_gb.columns = [get_info(col, "name") for col in base_hmms_gb.columns]
base_hmms_gb = base_hmms_gb.set_index("revised_tax").fillna(0)

In [ ]:
row_colors = pd.Series(base_hmms_gb.index).map(mapping).map(tax2color).fillna("white")
sns.set(font_scale=1)
sns.clustermap(base_hmms_gb,linewidths=1, cmap="Blues", row_cluster=True, figsize=(20,20), row_colors=list(row_colors))
plt.xticks(rotation=45, ha="right")
#plt.yticks(fontsize=0)
plt.ylabel("")
plt.xlabel("")
plt.show()

### metabolic profiles through dimensionality reduction

In [ ]:
# start with phylum level

genome_threshold = 8
trait_threshold = 3

# try filtering min # of representatives and generics
lineages_to_keep = []
for phylum in base["revised_tax"].unique():
    if list(base["revised_tax"]).count(phylum) > genome_threshold and\
        phylum not in ["Microgenomates", "Parcubacteria"]:
        lineages_to_keep.append(phylum)

# generate trait counts
traits_to_keep = []
for col in list(base_hmms_clean.columns):
    if col not in ["name", "revised_tax"]:
        if base_hmms_clean[col].sum() > trait_threshold:
            traits_to_keep.append(col)

print(len(lineages_to_keep), len(traits_to_keep))

#define agg dict
agg_dict = {}
for trait in traits_to_keep:
    agg_dict[trait] = "sum"
agg_dict["name"] = "count"

# regenerate base_hmms_gb applying filters
filtered = base_hmms_clean[base_hmms_clean["revised_tax"].isin(lineages_to_keep)][["name", "revised_tax"] + traits_to_keep]
fgb = filtered.groupby("revised_tax",as_index=False).aggregate(agg_dict)
for trait in traits_to_keep:
    fgb[trait] = fgb.apply(lambda x: x[trait]/float(x["name"]), axis=1)
fgb.pop("name")
fgb.columns = [get_info(col, "name") for col in fgb.columns]
fgb = fgb.set_index("revised_tax").fillna(0)

In [ ]:
#calculate distance matrix
brayD = ep.distance(fgb, method='bray', transform='1')
# then use skbio to do pcoA
results = pcoa(brayD)
results.proportion_explained.head()

In [ ]:
# add metadata back to results and custom plot
pc_results = pd.DataFrame(results.samples)
pc_results["name"] = fgb.index
pc_results["group"] = pc_results["name"].apply(lambda x: mapping[x])
pc_results["lineage_size"] = pc_results["name"].apply(lambda x: list(base["revised_tax"]).count(x))

sns.set(font_scale=1.25)
sns.set_style("ticks")
#sns.lmplot("PC1", "PC2", data=pc_results, hue="group", \
    #scatter_kws={"s":100, "linewidth":0.5, "edgecolors":"grey", "alpha":1}, fit_reg=False, size=7, aspect=1.5, palette=tax2color, legend=False)
kws = dict(linewidth=.5, edgecolor="black")
sns.relplot("PC2", "PC1", data=pc_results, size="lineage_size", hue="group", \
    palette=tax2color, height=10, aspect=.9, sizes=(50,400),**kws)

for line in range(0,pc_results.shape[0]):
    if (list(pc_results["PC1"])[line] > -10) or (list(pc_results["PC2"])[line] < -0.12) or (list(pc_results["PC2"])[line] > 0.0):
        plt.text(list(pc_results["PC2"])[line]+0.008, list(pc_results["PC1"])[line]-0.003, str(list(pc_results["name"])[line]), horizontalalignment='left',size=10, color='grey')
            
#plt.legend(loc="lower right")
plt.ylabel("PC1 (%s variance)" %(results.proportion_explained["PC1"]).round(2))
plt.xlabel("PC2 (%s variance)" %(results.proportion_explained["PC2"]).round(2))
#plt.yscale("log")
plt.savefig(rootdir + "/figures/lineage_clustering.svg", format="svg")

# building a metabolic reference set

We can extract a well-sampled set of reference sequences for metabolisms of interest from BAC175.

In [ ]:
cmdir(rootdir + "/bac175/metabolic_hmm_results")

In [ ]:
def run_metabolic_hmm(hmm_name):
    
    basename = os.path.basename(hmm_name).replace(".hmm", "")
    result_file = rootdir + "/bac175/metabolic_hmm_results/" + basename + ".results"
    call = "hmmsearch --cpu 10 --tblout " + result_file + " " + hmm_name + " " + rootdir + "/bac175/Bacteria175.cleaned.faa"
    sp.call(call, shell=True)
    #print(call)

total = 1
for source in glob.glob(rootdir + "/metabolism/hmms/by-source/*"):
    for hmm in glob.glob(source + "/*"):
        # only run for traits of interest
        ref_traits = traits + merged_traits_values
        if os.path.basename(hmm).split(".")[0] in ref_traits:
            run_metabolic_hmm(hmm)
            print('hmms run: [%d]\r'%total, end="")
            total += 1

For these genomes, just use noise cutoff or published cutoff (automated).

In [ ]:
cutoffs = {}

for key, row in hmm_metadata.iterrows():
    
    if row["noise_cutoff"] != "none":
        cutoff = row["noise_cutoff"]
    elif row["published_cutoff"] != "none":
        cutoff = row["published_cutoff"]
    else:
        cutoff = row["manual_cutoff"]
    cutoffs[row["hmm_name"]] = cutoff

In [ ]:
## READ IN RESULTS

ref_metabolic_hmm_results = {}

for hmm_result in glob.glob(rootdir + "/bac175/metabolic_hmm_results/*"):
    ref_metabolic_hmm_results[os.path.basename(hmm_result).split(".")[0]] = parse_hmm(hmm_result)
    
# then get filtered results
ref_metabolic_hmm_results_filtered = {}

for key in ref_metabolic_hmm_results.keys():
    
    try:
        cutoff = cutoffs[key]
    except:
        continue
    #print(key, cutoff)
    table = ref_metabolic_hmm_results[key]
    #table["position"] = table.index
        
    # only do if there are results and a cutoff
    if len(table) > 0:
        table_filt = table[table["score"] > float(cutoff)]
        table_subset = table_filt[["gene", "score","eval"]]
        table_subset["hmm"] = key
        ref_metabolic_hmm_results_filtered[key] = table_subset

In [ ]:
#concatenate all hmms
all_results = pd.concat(list(ref_metabolic_hmm_results_filtered.values()))
# then recast as long
all_results_long = all_results.pivot("gene", "hmm", "score").fillna(0)
# select best hit
all_results_long["best_hmm"] = all_results_long.idxmax(axis=1)
all_results_long["best_score"] = all_results_long.max(axis=1)
all_results_sub = all_results_long.reset_index()[["gene", "best_hmm", "best_score"]]

### then finish analysis

all_results_sub["name"] = all_results_sub["gene"].apply(lambda x: x.split("@")[0])
hmms_df = all_results_sub[["name", "gene", "best_hmm", "best_score"]]
# select best hit for each hmm within a bin
hmms_dfp = pd.pivot_table(hmms_df,index=["name","best_hmm"],columns="gene", values="best_score").fillna(0)
hmms_dfp["best_gene"] = hmms_dfp.idxmax(axis=1)
hmms_dfp = hmms_dfp.reset_index()[["name", "best_hmm", "best_gene"]]

In [ ]:
# pivot one more time
hmms_final = hmms_dfp.pivot("name", "best_hmm", "best_gene").fillna("None")
hmms_final = hmms_final.reset_index()

In [ ]:
all_genomes_metab = pd.concat([base_hmms_g, hmms_final]).fillna("None")

### fig. 1a - metabolisms

In [ ]:
colors = ['#d9d9d9','#1f78b4','#6a3d9a','#33a02c','#a6cee3','#fb9a99','#ff7f00','#fdbf6f','#cab2d6','#b15928']

In [ ]:
# compute hmm wise hit counts
sub = all_genomes_metab[~all_genomes_metab["name"].str.contains("BAC175")]
hit_counts = {}
for hmm in list(sub.columns):
    hits = 0
    for key,row in sub.iterrows():
        if row[hmm] != "None":
            hits += 1
    hit_counts[hmm] = hits

In [ ]:
groups_to_remove = ["Reverse TCA cycle", "Reverse TCA cycle", "Metal (Iron/Manganese) oxidation/reduction", "Gluconeogenesis",
                   "Arsenate reduction", "Methanol oxidation", "Sulfur oxidation", "Formaldehyde oxidation", "TCA cycle", "Nitric oxide reduction",
                   "FeFe hydrogenase", "CO oxidation", "Halogenated compounds breakdown","N2 fixation", "Sulfate reduction", "Sulfite reduction",
                   "Oxygen metabolism - cytochrome (quinone) oxidase, bd type"]

group_order={"Glycolysis": 1, "Pentose Phosphate Pathway: Non-Oxidative Branch": 2, "Pentose Phosphate Pathway: Oxidative Branch":3,
            "Pyruvate metabolism": 4, "Acetate/Lactate Metabolism": 5, "Nucleotide salvage pathway":6, "Other": 7, "Hydrogen/Sulfur Metabolism":8}

In [ ]:
metab_colors = {}

In [ ]:
# modify hmm_metadata for merged
mod_meta = hmm_metadata

for key in merged_traits:
    for hmm in merged_traits[key]:
        mod_meta = mod_meta.replace(hmm, key)

mod_meta_sub = mod_meta[["group", "gene", "hmm_name"]]
mod_meta_sub2 = mod_meta_sub[mod_meta_sub["hmm_name"].isin(traits)].drop_duplicates()     
mmf = mod_meta_sub2[~mod_meta_sub2["group"].isin(groups_to_remove)]

In [ ]:
# reassign groups
fixes = {"Fermentation": "Acetate/Lactate Metabolism", "Acetate metabolism": "Acetate/Lactate Metabolism", "Ni-Fe Hydrogenase": "Other", "Nitrite reduction": "Other", "Oxygen metabolism - cytochrome (quinone) oxidase, bo type": "Other"}
mmf["group"] = mmf["group"].apply(lambda x: x if x not in fixes else fixes[x])
# get hit counts
mmf["count"] = mmf["hmm_name"].apply(lambda x: hit_counts[x])

In [ ]:
# make hmm metadata dict
hmm_dict = {}

for index, row in mmf.iterrows():
    hmm_dict[row["hmm_name"]] = {"gene": row["gene"], "group": row["group"]}

In [ ]:
with open(rootdir + "/metabolism/itol/all_metabs.txt","w") as outfile:
    
    groupss = list(set(mmf["group"]))
    outfile.write("DATASET_BINARY\nSEPARATOR COMMA\nDATASET_LABEL,all_metabolism\nCOLOR,#ff0000\nFIELD_SHAPES," + \
        ("1,"*(len(mmf) + len(groupss)-1)).strip(",") + "\nFIELD_LABELS,")
    
    #print((len(hmm_subset) + len(groups)))
    field_labels = ""
    color_string = ""
    
    # get hit counts for each hmm
    test = mmf[["group", "count"]]
    #compute relative completeness
    test["frac"] = test["count"].apply(lambda x: x/float(max(mmf["count"])))
    tgb = test.groupby("group",as_index=False).aggregate({"frac":"mean"})
    hmm_sub2 = mmf.merge(tgb, on="group")
    hmm_sub2["order"] = hmm_sub2["group"].map(group_order)
    # get a sorted df
    sorted_df = hmm_sub2.sort_values(["order", "count"], ascending=[True,False])
    
    position = "None"
    for key, row in sorted_df.iterrows():
        if position != row["group"] and position != "None":
            field_labels += ","
            color_string += "#ffffff,"
        field_labels += hmm_dict[row["hmm_name"]]["gene"]+","
        color_string += colors[row["order"] % len(colors)] + ","
        position = row["group"]
        metab_colors[row["group"]] = colors[row["order"] % len(colors)]
        #print(row["group"], row["order"], colors[row["order"] % len(colors)])
    
    #print(len(field_labels.split(",")), len(color_string.split(",")))
 
    # write out blocks, removing final comma where necessary
    outfile.write(field_labels.strip(",") + "\n")
    outfile.write("FIELD_COLORS," + color_string.strip(",") + "\nDATA\n")
    
    # now go by genome
    for key, row in all_genomes_metab.iterrows():
        if "BAC175" in row["name"]:
            outfile.write(row["name"] + ",")
        else:
            # instead of revised to match old names
            outfile.write(row["final_tax"] + "_" + row["name"] + ",")
        data_block = ""
        # iterate through metabs in order
        position = "None"
        for keye, rowe in sorted_df.iterrows():
            if position != rowe["group"] and position != "None":
                data_block += "-1,"
            data_block += "1," if row[rowe["hmm_name"]] != "None" else "-1,"
            position = rowe["group"]
        outfile.write(data_block.strip(",") + "\n")
        #print(len(data_block.split(",")))

# trait depth + homoplasy metrics

### create input for R script

In [ ]:
cmdir(rootdir + "/metabolism/trait_analysis/")

In [ ]:
# iteratively write out binary matrix for all traits
base_hmms = base_hmms.drop_duplicates()
base_hmms["newname"] = base_hmms.apply(lambda x: x["final_tax"] + "_" + x["name"], axis=1)
base_hmms_sub = base_hmms[["newname"] + traits]

base_hmms_clean = base_hmms_sub[["newname"]]

total = 0
for trait in traits_sorted:
    if len(set(base_hmms_sub[trait])) != 1:
        base_hmms_clean[trait] = base_hmms_sub[trait].apply(lambda x: 1 if x != "None" else 0)
    
    print('traits processed: [%d]\r'%total,end="")
    total += 1

# add in dummy outgroup data
t = Tree(rootdir + "/trees/bac175_outgroup/rp16_concat.BAC175.final.pruned.treefile")
outs = [leaf.name for leaf in t if "BAC175" in leaf.name]
top_dict = {}
for item in outs:
    temp_dict = {}
    for trait in traits:
        temp_dict[trait] = 0
    top_dict[item] = temp_dict

outdf = pd.DataFrame.from_dict(top_dict, orient="index")
outdf = outdf.reset_index()
outdf.columns = ["newname"] + traits
merged = pd.concat([base_hmms_clean.drop_duplicates(), outdf])

# write out for R
merged.to_csv(rootdir + "/metabolism/trait_analysis/trait_table.csv", header=True)

In [ ]:
# associated viz for itol
for trait in traits:
    
    if ("TIGR") in trait or ("Pfam" in trait):
        out_path = groups[trait]["name"].replace(" ", "_").replace("/","_") + "_" + trait + ".txt"
    else: out_path = trait.replace("/","_") + ".txt"
    
    with open(rootdir + "/metabolism/itol/" + out_path,"w") as outfile:
        outfile.write("TREE_COLORS\nSEPARATOR TAB\nDATA\n")
        for key, row in base_hmms_clean.iterrows():
            if row[trait] != 0:
                outfile.write(row["newname"]+ "\trange\t" + "#b3e2cd" + "\t" + trait + "\n")

### analyze results

IMPORTANT: RUN R SCRIPT cpr-phylo-R-public.R to generate results used below.

In [ ]:
td = pd.read_csv(rootdir + "/metabolism/trait_analysis/bac175_consentrait_results.csv")
td["group"] = td["trait"].apply(lambda x: groups[x.replace(".", "/")]["process"])
td["gene"] = td["trait"].apply(lambda x: groups[x.replace(".", "/")]["name"])
tdm = td[["tree","trait","group", "gene", "min_fraction", "pval", "clade", "mean_depth"]]
tdm["desc"] = tdm.apply(lambda x: x["trait"] + " - " + x["gene"], axis=1)
tdm["sig"] = tdm["pval"].apply(lambda x: x < 0.05)
# mean sort for boxplot
order = list(tdm.groupby("trait").aggregate({"mean_depth": "median"}).sort_values("mean_depth", ascending=False).reset_index()["trait"])

In [ ]:
# lets focus on rp 16 at a certain threshold
thresh = 0.9
sub = tdm[(tdm["min_fraction"]==thresh) & (tdm["tree"]=="rp16")]
order = list(sub.groupby("desc").aggregate({"mean_depth": "median"}).sort_values("mean_depth", ascending=False).reset_index()["desc"])
plt.figure(figsize=(30,8))
sns.set(font_scale=1.75)
sns.set_style("white")
sns.boxplot("desc", "mean_depth", data=sub, palette={True:"red", False:"pink"}, order=order, linewidth=0.5, hue="sig")
sns.stripplot("desc", "mean_depth", data=sub, jitter=True, size=5, linewidth=1, order=order, color="black")
plt.xticks(rotation=45, ha="right")
plt.title("phylogenetic depth of positive clades, by trait (min_fraction = %.2f)" %(thresh))
plt.xlabel("")
plt.ylabel("phylogenetic depth")
plt.show()

In [ ]:
# process homoplasy info
hm = pd.read_csv(rootdir + "/metabolism/trait_analysis/bac175_ci_results.csv")
# using formula from annotree paper
hm["patchiness"] = hm.apply(lambda x: math.log(x["ci"])/float(math.log(x["family_size"])) \
    if x["family_size"]>1 else None, axis=1)
# filter out small fam sizes
#hm = hm[hm["family_size"]>3]

In [ ]:
# set display classes
display_groups = list(group_order.keys()) + ["Ni-Fe Hydrogenase", "Nitrite reduction", "Oxygen metabolism - cytochrome (quinone) oxidase, bo type"]
# add colors
new_cols = ['#fdbf6f','#cab2d6','#b15928']

count=0
for metab in display_groups:
    
    if metab not in list(group_order.keys()):
        
        metab_colors[metab] = new_cols[count]
        count+=1

In [ ]:
### plot td vs homoplasy for one min_frac
td_sub = td[td["min_fraction"]==thresh].groupby(["tree", "trait"], as_index=False).aggregate({"mean_depth":"mean"})
both = pd.merge(td_sub, hm, on=["tree","trait"])
both["group"] = both["trait"].apply(lambda x: groups[x.replace(".", "/")]["process"])
both["group"] = both["group"].apply(lambda x: x if x not in ["Acetate metabolism", "Fermentation"] else "Acetate/Lactate Metabolism")
#both_meta = pd.merge(both, hmm_metadata, left_on="trait", right_on="hmm_name")
# filter down to metabs of interest
both_meta_sub = both[both["group"].isin(display_groups)]
for tree in both["tree"].unique():
    
    sns.set(font_scale=1)
    sns.set_style("ticks")
    kws = dict(linewidth=.5, edgecolor="black")
    sns.relplot("mean_depth", "patchiness", data=both_meta_sub[both_meta_sub["tree"]==tree], size="family_size", hue="group", \
                palette=metab_colors, alpha=0.8, height=6, aspect=1.1, sizes=(50,400), **kws, legend="brief")
    #l = plt.legend(loc="best")
    #l.set_title('Metabolism')
    # taking out one enzyme
    #plt.xlim([0,0.7])
    #plt.xscale("log")
    plt.xlabel("mean phylogenetic depth")
    #plt.title("evolutionary profiles on " + tree + " tree")
    if tree == "rp16":
        plt.savefig(rootdir + "/figures/all_evol_profiles.svg", format="svg")
    plt.show()

In [ ]:
for group in both_meta_sub["group"].unique():
    
    if group in ["Glycolysis"]:
        subset = both_meta_sub[(both_meta_sub["tree"]=="rp16")]
        subset["in_gene_set"] = both_meta_sub["group"].apply(lambda x: x == group)
        sns.set(font_scale=1)
        sns.set_style("ticks")
        kws = dict(linewidth=.5, edgecolor="black")
        sns.relplot("mean_depth", "patchiness", data=subset, size="family_size", hue="in_gene_set", \
                    palette={True:metab_colors[group], False:"white"}, alpha=0.8, height=6, aspect=1.1, sizes=(50,400), **kws, legend="brief")
        #l = plt.legend(loc="best")
        #l.set_title('Metabolism')
        # taking out one enzyme
        #plt.xlim([0,0.7])
        #plt.xscale("log")
        plt.xlabel("mean phylogenetic depth")
        plt.savefig(rootdir + "/figures/glycolysis_evol_profiles.svg", format="svg")
        plt.show()

# create gene trees

### pull seqs, align, treebuild

In [ ]:
# define traits to focus on
traits_sub = hmm_metadata[(hmm_metadata["group"].str.contains("Glycolysis"))]["hmm_name"].to_list()
# traits for which we want archaeal references added
arch_traits = ["PF06560", "TIGR02128", "TIGR00306", "TIGR00419"]

In [ ]:
basedir = rootdir + "/metabolism/gene_trees/filtered_sequences/"

cmdir(rootdir + "/metabolism/gene_trees/")
cmdir(basedir)

def get_arch_refs(trait):
    
    # run hmmsearch w/ threshold
    db = "TIGRFAMs" if "TIGR" in trait else "Pfam"
    call = hpath + " --cut_nc --cpu 6 --tblout " + basedir + trait + ".ARC.results " + \
        rootdir + "/metabolism/hmms/by-source/" + db + "/" + trait + ".hmm " + \
        rootdir + "/metabolism/Archaea300.mod.faa"
    sp.call(call, shell=True)
    
    # read in results + write out names
    with open(basedir + trait + ".ARC.names", "w") as outfile:
        for key, row in parse_hmm(basedir + trait + ".ARC.results").iterrows():
            outfile.write(row["gene"] + "\n")
    
def extract_pull(trait, basedir, archaeal):
    
    gene_list = [gene for gene in base_hmms[trait] if gene != "None"] + \
        [gene for gene in hmms_final[trait] if gene != "None"]
    
    with open(basedir + trait + ".names", "w") as q_out:
        for gene in gene_list:
            q_out.write(gene + "\n")

    call = pspath + " -i " + rootdir + "/proteins/all_proteins.faa" + \
        " -n " + basedir + trait + ".names >> " + basedir + trait + ".concat.faa"
    call2 = pspath + " -i " + rootdir + "/bac175/Bacteria175.mod.faa" + \
        " -n " + basedir + trait + ".names >> " + basedir + trait + ".concat.faa"
    
    sp.call(call,shell=True)
    sp.call(call2, shell=True)
    
    if archaeal == True:
        
        # create reference sequences
        get_arch_refs(trait)
        # add to concat file
        call3 = pspath + " -i " + rootdir + "metabolism/Archaea300.mod.faa" + \
            " -n " + basedir + trait + ".ARC.names >> " + basedir + trait + ".concat.faa"
        sp.call(call3, shell=True)
        
with open(basedir + "wrapper.sh", "w") as wrapper:

    for trait in traits_sub:
        
        # only for those w/ valid hits
        if trait in base_hmms.columns.to_list() and trait in hmms_final.columns.to_list():
            
            if trait in arch_traits:
                extract_pull(trait, basedir, True)
            
            else:
                extract_pull(trait, basedir, False)

            # write out align + tree calls to wrapper
            aln_call = mpath + " --thread 8 --anysymbol --reorder " + basedir + trait + ".concat.faa > " + basedir + trait + ".concat.mafft"
            wrapper.write(aln_call + "\n")

<b> Then chmod +x and run the wrapper manually. </b>

Next, use Geneious or other method to strip columns with 95% or more gaps in each single gene alignment, creating *.stripped.mafft* versions for each alignment used in the next step.

### run trees on iqtree

N.B. Uses qsub, modify below command if necessary.

In [ ]:
# and run the trees
for aln in glob.glob(rootdir + "/metabolism/gene_trees/filtered_sequences/*concat*stripped.mafft"):
    
    basename = os.path.basename(aln).split(".")[0]
    call = "echo '" + iqpath + " -s " + aln + " -m TEST -st AA -bb 1500 -nt 48 -pre " + aln.split(".")[0] + "' | qsub -V -N " + basename
    sp.call(call, shell=True)
    #print(call)

### create mod trees + itol files

In [ ]:
revised_tax = {}
for key, row in base.iterrows():
    revised_tax[row["name"]] = row["revised_tax"]
    
# add taxes not already present
for cat in base["revised_tax"].unique():
    if cat not in mapping:
        mapping[cat] = "other"

In [ ]:
cmdir(rootdir + "/metabolism/gene_trees/itol/")
cmdir(rootdir + "/metabolism/gene_trees/mod_trees/")

for tree in glob.glob(rootdir + "/metabolism/gene_trees/filtered_sequences/*treefile"):
    
    basename = os.path.basename(tree).split(".")[0]
    t = Tree(tree)
    with open(rootdir + "/metabolism/gene_trees/itol/" + basename + ".itol.txt", "w") as outfile:
        outfile.write("TREE_COLORS\nSEPARATOR TAB\nDATA\n")
        for leaf in t:
            # remove geneious artifact
            leaf.name = leaf.name.replace("__stripped_","")
            if "BAC175" in leaf.name:
                outfile.write(leaf.name + "\trange\t#595957\t" + basename + "\n")
            elif "ARC" in leaf.name:
                outfile.write(leaf.name + "\trange\t#1c1c1b\t" + basename + "\n")
            else:
                #print(leaf.name)
                bin = retrieve_bin(scaffold(leaf.name), scaf2bin)
                #print(bin)
                leaf.name = bin + "_" + leaf.name
                outfile.write(leaf.name + "\trange\t" + tax2color[mapping[revised_tax[bin]]] + "\t" + basename + "\n")
        t.write(outfile=rootdir + "/metabolism/gene_trees/mod_trees/" + basename + ".mod.tre",format=2)

## metabolic case studies

### hydrogenase 3b tree

We start with a trimmed alignment of CPR hydrogenase plus a dereplicated reference set.

In [ ]:
# modify alignment file
with open(rootdir + "/metabolism/case_studies/hyd.concat.cleaned.mafft", "w") as clean_aln:
    
    for record in SeqIO.parse(open(rootdir + "/metabolism/case_studies/hyd.concat.stripped.mafft"), "fasta"):
        
        if ":" in record.description:
            new_desc = record.description.replace(":", "_")
        else:
            new_desc = record.description
        
        clean_aln.write(">" + new_desc + "\n" + str(record.seq) + "\n")

In [ ]:
# and run the trees
for aln in glob.glob(outdir + "/hyd.concat.cleaned.mafft"):
    
    basename = os.path.basename(aln).split(".")[0]
    call = iqpath + " -s " + aln + " -m TEST -st AA -bb 1500 -nt 6 -pre " + aln.split(".")[0] + ".concat.cleaned."
    sp.call(call, shell=True)
    #print(call)

In [ ]:
so_many_colors = ["#e6194B","#3cb44b","#ffe119","#4363d8","#f58231","#911eb4","#ffa500","#FFD280",
                  "#42d4f4","#f032e6","#bfef45","#fabebe","#469990","#e6beff","#9A6324","#fffac8","#800000","#aaffc3","#808000","#ffd8b1",
                  "#000075","#a9a9a9","#ffffff","#000000"]

In [ ]:
t = Tree(rootdir + "/metabolism/case_studies/hyd.concat.cleaned.tre")

groups = []

with open(rootdir + "/metabolism/case_studies/sulf_forms.itol.txt","w") as outfile:
    outfile.write("TREE_COLORS\nSEPARATOR TAB\nDATA\n")
    for leaf in t:
        if "Form_V" in leaf.name:
            name = leaf.name.replace("Form_V", "Group_1h/5")
        else:
            name = leaf.name
        if "Group" in name:
            group = " ".join(name.split("_")[-2:])
            if group not in groups:
                groups.append(group)
            color = so_many_colors[groups.index(group)]
            outfile.write(leaf.name + "\trange\t" + color + "\t" + group + "\n")

### additional hmms

In [ ]:
## READ IN RESULTS 

hyd_hmm_results = {}

for hmm_result in glob.glob(rootdir + "/metabolism/case_studies/*results"):
    hyd_hmm_results[os.path.basename(hmm_result).split(".")[0]] = parse_hmm(hmm_result)

In [ ]:
for result in hyd_hmm_results.keys():   
            
    table = hyd_hmm_results[result]
    table["position"] = table.index
    table["significant"] = table["eval"].apply(lambda x: x < 0.05)

    #THEN PLOT
    sns.set(font_scale=1.5)
    sns.set_style("ticks")
    plt.figure(figsize=(10,3)) 

    sns.regplot("position", "score", data=table, fit_reg=False, scatter_kws={'s':40}, color="blue")
    plt.xlabel("rank")
    plt.xticks()
    ax2 = plt.twinx()
    ax2.set_ylim(0, 0.05)
    ax2.grid(False)
    sns.regplot("position", "eval", data=table, fit_reg=False, scatter_kws={'s':40}, ax=ax2, color="orange")
    plt.title("%s" %(result))
    plt.show()

In [ ]:
for result in hyd_hmm_results.keys():
    
    with open(rootdir + "/metabolism/case_studies/" + result + ".names", "w") as outfile:
        
        table = hyd_hmm_results[result]
        for key,item in table.iterrows():
            if item["eval"] < 0.05:
                outfile.write(item["gene"]+"\n")
                
    with open(rootdir + "/metabolism/case_studies/" + result + ".itol.txt", "w") as itol:
        
        itol.write("DATASET_SIMPLEBAR\nSEPARATOR COMMA\nDATASET_LABEL,hmm_score\nSHOW_VALUE,1\nCOLOR,#ff0000\nDATA\n")
        table = hyd_hmm_results[result]
        for key,item in table.iterrows():
            if item["eval"] < 0.05:
                itol.write(item["gene"] + "," + str(item["score"]) + "\n")

In [ ]:
sulf_cutoffs = {"Fer4_22": 34.4, "Oxidored_q6": 0, "NAD_binding_1":0}

sulf_filt = {}

for key in hyd_hmm_results.keys():
    
    table = hyd_hmm_results[key]
    filt_table = table[(table["eval"] < 0.05) & (table["score"] > sulf_cutoffs[key])]
    sulf_filt[key] = filt_table
    
    # write out names to match with protein fams
    with open(rootdir + "/metabolism/case_studies/" + key + ".filtered.names.txt", "w") as outfile:
        
        for keye, rowe in filt_table.iterrows():
            outfile.write(rowe["gene"] + "\n")

In [ ]:
hyd = base_hmms[["name","Hydrogenase_Group_3b"]]
hyd = hyd[hyd["Hydrogenase_Group_3b"]!="None"]
hyd["bin"] = hyd["Hydrogenase_Group_3b"].apply(lambda x: retrieve_bin(scaffold(x), scaf2bin))

In [ ]:
for key in sulf_filt.keys():
    
    table = sulf_filt[key]
    table["bin"] = table["gene"].apply(lambda x: retrieve_bin(scaffold(x), scaf2bin))
    table_sub = table[["bin", "gene"]]
    table_sub.columns = ["bin",key]
    hyd = hyd.merge(table_sub, on="bin", how="left").fillna("None")

In [ ]:
def get_proximity(row):
    
    return len(set(scaffold(row[x]) for x in ["Hydrogenase_Group_3b","Fer4_22","Oxidored_q6", "NAD_binding_1"]))

# get # unique scafs
hyd["scaf_count"] = hyd.apply(lambda x: get_proximity(x), axis=1)
# sort low to hihgh
hyd_sort = hyd.sort_values(["scaf_count", "bin"], ascending=[True,True])
# remove duplicates w/ more scaffolds - taking best
hyd_drep = hyd_sort.drop_duplicates(subset=["name", "Hydrogenase_Group_3b"], keep="first")

In [ ]:
# write out shapes for hyd, asrAB
with open(rootdir + "/metabolism/case_studies/sulf_hmms.txt","w") as outfile:
    outfile.write("DATASET_BINARY\nSEPARATOR COMMA\nDATASET_LABEL,sulfhydrogenase_components\nCOLOR,#ff0000\nFIELD_SHAPES,2,2,2\nFIELD_LABELS,")
    outfile.write("Oxidored_q6,Fer4_22,NAD_binding_1\n")
    outfile.write("FIELD_COLORS,#1b9e77,#d95f02,#7570b3\nDATA\n")
    
    for key, row in hyd_drep.iterrows():
        outfile.write(row["Hydrogenase_Group_3b"] + ",")
        block = ""
        for gene in ["Oxidored_q6", "Fer4_22", "NAD_binding_1"]:
            # same scaffold
            if scaffold(row[gene]) == scaffold(row["Hydrogenase_Group_3b"]):
                block += "1,"
            # different scaffold
            elif row[gene] != "None":
                block += "0,"
            # none
            else: block += "-1,"
        outfile.write(block.strip(",") + "\n")

In [ ]:
with open(rootdir + "/metabolism/case_studies/sulf_hmms_tax.itol.txt","w") as outfile:
    outfile.write("DATASET_COLORSTRIP\nSEPARATOR TAB\nDATASET_LABEL\ttaxonomy\nDATA\n")
    for key, row in base_hmms.iterrows():
        if row["Hydrogenase_Group_3b"] != "None":
            htype = "3b"
        elif row["Hydrogenase_Group_4"] != "None":
            htype = "4"
        else: htype = None
        if htype and row["revised_tax"] in mapping:
            color = tax2color[mapping[row["revised_tax"]]]
            outfile.write(row["Hydrogenase_Group_" + htype]+ "\t" + color + "\n")

### genomic context analysis

In [ ]:
# build contig db for all proteins
contig_db = {}

for record in SeqIO.parse(open(rootdir + "/proteins/all_proteins.faa"), "fasta"):
    m = re.search("(\S+) # ([0-9]+) # ([0-9]+) # ([1-]+) .+", record.description)
    scaf = scaffold(m.group(1))
    if scaf not in contig_db:
        contig_db[scaf] = [m.group(1)]
    else:
        contig_db[scaf].append(m.group(1))

In [ ]:
# define foci and look in neighborhood
foci = list(base_hmms.query("Hydrogenase_Group_3b != 'None'")["Hydrogenase_Group_3b"])
# define search radius
radius = 20

neighbors = {}
for focus in foci:
    gene_array = contig_db[scaffold(focus)]
    # define span
    upper_bound = min(len(gene_array)-1, gene_array.index(focus) + radius)
    lower_bound = max(0, gene_array.index(focus) - radius)
    # add results to dict
    for i in range(lower_bound, upper_bound+1):
        # don't include focus
        if i != gene_array.index(focus):
            neighbors[gene_array[i]] = {"position": i-gene_array.index(focus), "focus":focus}

neighbor_df = pd.DataFrame.from_dict(neighbors, orient="index")
neighbor_df = neighbor_df.reset_index()
neighbor_df.columns = ["gene", "position", "focus"]

In [ ]:
#adjust position by strand orientation
neighbor_df["gene_strand"] = neighbor_df["gene"].apply(lambda x: strand_dict[x])
neighbor_df["focus_strand"] = neighbor_df["focus"].apply(lambda x: strand_dict[x])
neighbor_df["adj_position"] = neighbor_df.apply(lambda x: int(x["position"])*int(x["gene_strand"]), axis=1)

In [ ]:
# adjust for few cases where hydrogenase is split
table = metabolic_hmm_results_filtered["Hydrogenase_Group_3b"]
table["scaf"] = table["gene"].apply(lambda x: scaffold(x))
counts = table["scaf"].value_counts().reset_index()
dups = counts[counts["scaf"]>1]["index"].to_list()

def correct_dups(row):
    
    if scaffold(row["focus"]) in dups:
        
        # get gene id of second piece
        pieces = table[table["gene"].str.contains(scaffold(row["focus"]))]["gene"].to_list()
        dropped = pieces[pieces != row["focus"]]
        # get relationship between dropped and focus
        rel = neighbor_df[neighbor_df["gene"]==dropped].iloc[0]["adj_position"]
        
        # modify upstream/downstream genes accordingly
        if rel > 0 and row["adj_position"] > 0:
            return row["adj_position"] - 1
        elif rel < 0 and row["adj_position"] < 0:
            return row["adj_position"] - 1
        else: return row["adj_position"]
    
    else:
        return row["adj_position"]

In [ ]:
neighbor_df["cor_position"] = neighbor_df.apply(correct_dups, axis=1)
# get rid of extra pieces from table
neighbor_df = neighbor_df[neighbor_df["cor_position"] != 0]

In [ ]:
# write out genes for protein clustering
cmdir(rootdir + "/proteins/protein_clustering/")

with open(rootdir + "/proteins/protein_clustering/neighbors.txt", "w") as outfile:
    
    for key, row in neighbor_df.iterrows():
        outfile.write(row["gene"] + "\n")

call = pspath + " -n " + rootdir + "/proteins/protein_clustering/neighbors.txt -i " + \
        rootdir + "/proteins/all_proteins.faa > " + \
        rootdir + "/proteins/protein_clustering/neighbors.faa"
sp.call(call, shell=True)

In [ ]:
# start by subfamily clustering
call = pcpath + "/subfamilies.py --output-directory " + \
    rootdir + "/proteins/protein_clustering/output/ --cpu 6 " + \
    rootdir + "/proteins/protein_clustering/neighbors.faa"
sp.call(call, shell=True)
#print(call)

In [ ]:
# then do hmm-hmm comparison to generate families
call1 = pcpath + "/hhblits.py --cpu 6 " + rootdir + "/proteins/protein_clustering/output/config.json"
print(call1)
#sp.call(call1, shell=True)
call2 = pcpath + "/runningMclClustering.py --coverage 0.50 --fasta --cpu 6 " + rootdir + "/proteins/protein_clustering/output/config.json"
#sp.call(call2, shell=True)
print(call2)

In [ ]:
# integrate with family stuff (see miscellaneous)
fams = {}

count = 1
for line in open(rootdir + "/proteins/protein_clustering/output/orf2family.tsv").readlines():
    # skip headers
    if count != 1:
        splt = line.strip().split("\t")
        fams[splt[0]] = splt[1]
    count +=1

# map subfams to neighbors
neighbor_df["fam"] = neighbor_df["gene"].apply(lambda x: fams[x] if x in fams else "None")

In [ ]:
# add in phylogenetic type
types = {}

for file in glob.glob(rootdir + "/metabolism/case_studies/hyd_*.txt"):
    for line in open(file).readlines():
        types[line.strip()] = os.path.basename(file).split(".")[0]

neighbor_df["type"] = neighbor_df["focus"].apply(lambda x: types[x] if x in types else "None")

In [ ]:
fam_colors = {"Oxidored_q6": "#1b9e77","Fer4_22":"#d95f02","NAD_binding_1":"#7570b3"}

In [ ]:
# correlate subfams + hmms
family2hmm = {}

top_families = list(neighbor_df["fam"].value_counts()[0:15].index)
top_families = [top for top in top_families if top !="None"]

for family in top_families:
    
    genes = neighbor_df[neighbor_df["fam"]==family]["gene"].to_list()

    for hmm in sulf_filt.keys():
        db = sulf_filt[hmm]["gene"].to_list()
        overlap = set(genes).intersection(set(db))
        if len(overlap)/float(len(genes)) > 0.5:
            family2hmm[family] = hmm

In [ ]:
# define subset and groupby
for phy in neighbor_df["type"].unique():
    
    if phy != "None":
        phy_df = neighbor_df[(neighbor_df["type"]==phy)  & (neighbor_df["gene_strand"]==neighbor_df["focus_strand"])]
        top_families = list(phy_df["fam"].value_counts()[0:7].index)
        top_families = [top for top in top_families if top !="None"]
        
        # make color palette
        family2color = {}
        for family in top_families:
            if family in family2hmm:
                family2color[family] = fam_colors[family2hmm[family]]
            else:family2color[family] = "#d3d3d3"
        ng = phy_df[(phy_df["fam"].isin(top_families))].groupby(["fam", "cor_position"], as_index=False).count()
        ng = ng[["fam", "cor_position", "focus"]]
        ng.columns = ["fam", "cor_position", "count"]
        # restrict radius
        ng = ng[abs(ng["cor_position"])<=10]
        
        sns.set(font_scale=1)
        sns.set_style("white", {"axes.edgecolor": "0.8"})
        kws = dict(linewidth=.5, edgecolor="black")
        g = sns.relplot("cor_position", "fam", data=ng, size="count", hue="fam",palette=family2color, alpha=1, height=2.5, aspect=3, sizes=(50,500), **kws, legend="brief")
        #for i in range(min(ng["cor_position"])-1,max(ng["cor_position"]+1)):
        for i in range(-10,11,1):
            if i >= 0 and i < 6:
                plt.axhline(i, color='grey', linestyle='-', lw=0.5,zorder=0)
            if i==0:
                plt.axvline(i, color='black', linestyle='-', lw=2,zorder=1)
            else:
                plt.axvline(i, color='grey', linestyle='-', lw=0.5,zorder=0)
        
        def round(x):
            return math.ceil(x / 2.) * 2
        
        #g.set(xticks=[i for i in range(round(min(ng["cor_position"])-1),round(max(ng["cor_position"])+1),2)])
        g.set(xticks=[i for i in range(-10,11,2)])
        plt.ylabel("")
        plt.xlabel("relative gene position")
        sns.despine(left=False, bottom=True, top=True, right=False)
        plt.savefig(rootdir + "/figures/" + phy + "_context.svg", format="svg")
        
        plt.show()

### hyd small subunit tree

In [ ]:
# create alignment
call = mpath + " --thread 8 --reorder " + rootdir + "/proteins/protein_clustering/output/familiesFasta/fam019.fa > " + \
    rootdir + "/metabolism/case_studies/hyd_SSU.mafft"
print(call)

N.B. Uses qsub, modify below command if necessary. Use Geneious to strip gene alignment as described above and create new *stripped* file.

In [ ]:
# and run the trees
for aln in glob.glob(rootdir + "/metabolism/case_studies/*SSU*stripped.mafft"):
    
    basename = os.path.basename(aln).split(".")[0]
    call = "echo '" + iqpath + " -s " + aln + " -m TEST -st AA -bb 1500 -nt 48 -pre " + aln.split(".")[0] + "' | qsub -V -N " + basename
    #sp.call(call, shell=True)
    print(call)

In [ ]:
# create itol 
with open(rootdir + "/metabolism/case_studies/hyd_SSU.itol.txt", "w") as outfile:
    
    outfile.write("DATASET_ALIGNMENT\nSEPARATOR COMMA\nDATASET_LABEL,hyd_SSU_aln\nCOLOR,#ff0000\nCUSTOM_COLOR_SCHEME,MY_SCHEME_1,A=#d2d0c9,M=#d2d0c9,I=#d2d0c9,L=#d2d0c9,V=#d2d0c9,P=#746f69,G=#746f69,C=#746f69,F=#d0ad16,Y=#d0ad16,W=#d0ad16,S=#34acfb,T=#34acfb,N=#34acfb,-=#ffffff,Q=#34acfb,R=#34fb54,K=#34fb54,H=#34fb54,D=#fb4034,E=#fb4034\nDATA\n")
    
    for record in SeqIO.parse(open(rootdir + "/metabolism/case_studies/hyd_SSU.stripped.mafft"), "fasta"):
        outfile.write(">" + record.description.split(" ")[0] + "\n" + str(record.seq) + "\n")

In [ ]:
# write itol ranges
with open(rootdir + "/metabolism/case_studies/hyd_SSU.ranges.txt", "w") as out:
    
    out.write("TREE_COLORS\nSEPARATOR TAB\nDATA\n")
    for key, row in neighbor_df.iterrows():
        
        if row["fam"] == "fam019":
            if row["type"] == "hyd_1":
                out.write(row["gene"] + "\trange\tlightgrey\thyd_SSU\n")
            else: out.write(row["gene"] + "\trange\twhite\thyd_SSU\n")